In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
import math
import statistics
import os
from sqlalchemy import create_engine, text

In [2]:
ENERGY_UNIT = 'MWh'
CAPACITY_UNIT = 'MWh'
COST_UNIT = '$'
EMISSION_UNIT = 't'

In [3]:
#scenario = "base_thin" # 
#current_period = 2040 # Part of dubeg suite.
#folder = "pg_inputs_26z_wacc_5_no_ramp/settings"
scenario = 'current_policies_20_week'
scenario = "base_20_week"
folder = "settings-atb2023"
MIN_CAP_THRESHOLD = 10 # MW

In [4]:
cwd = Path.cwd()

#settings_path = (
#    cwd.parent / "Example-Settings" / folder
#)
settings_path = (
    cwd.parent / "Git-Results-Comparison" / "MIP_results_comparison" / "case_settings" / "26-zone" / folder 
)

results_directory = str(settings_path) + '/Computed_Data_' + scenario + '/'
#results_directory = str(settings_path) + '/Computed_Data_' + scenario + '_' + str(current_period) + '/' # Part of dubeg suite.

sql_directory = str(settings_path) + '/SQL_Files_' + scenario + '/'
#sql_directory = str(settings_path) + '/SQL_Files_' + scenario + '_' + str(current_period) + '/' # Part of dubeg suite.

isExist = os.path.exists(sql_directory)
if not isExist:
    os.makedirs(sql_directory)
temoa_tables_dict = {}



#### Read in csv files

Note:
- "new_gens" represents all new generator options for each planning year.
- "all_gens" are all existing generators AND the new gens for the first planning year only.
    
    

In [5]:
pg_tx =  pd.read_csv(results_directory + 'transmission.csv', index_col=0)
pg_new_gens = pd.read_csv(results_directory + 'new_gens.csv')   
pg_load_profile = pd.read_csv(results_directory + 'reduced_load_profile.csv')
pg_all_gens = pd.read_csv(results_directory + 'all_gens.csv')
pg_existing_gens = pd.read_csv(results_directory + 'existing_gens.csv')
pg_time = pd.read_csv(results_directory + 'time.csv')
pg_fuel_price = pd.read_csv(results_directory + 'fuels_price.csv')
pg_fuel_co2 = pd.read_csv(results_directory + 'fuels_co2.csv')
pg_demand = pd.read_csv(results_directory + 'elec_demand_all_years.csv')
pg_capres = pd.read_csv(results_directory + 'regional_cap_res.csv')
pg_policies = pd.read_csv(results_directory + 'policies.csv')
#pg_policies = pg_policies[pg_policies.case_id == scenario]
pg_policies = pg_policies[(pg_policies.case_id == scenario) | (pg_policies.case_id == 'all')]


In [6]:

pg_policies

,Unnamed: 0,case_id,year,region,ESR_1,ESR_2,ESR_3,ESR_4,ESR_5,ESR_6,...,ESR_11,ESR_12,ESR_13,ESR_14,ESR_15,ESR_16,CO_2_Cap_Zone_1,CO_2_Max_Mtons_1,copy_case_id,Note
0,0,all,2027,BASN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,873.0,NaN,halfway between 2023 (1560 MMT according to EP...
1,1,all,2027,CANO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2,all,2027,CASO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,3,all,2027,FRCC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,4,all,2027,ISNE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,151,all,2050,SRCE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
152,152,all,2050,SRSE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
153,153,all,2050,SRSG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
154,154,all,2050,TRE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [7]:
pg_existing_gens.model_year.unique()
for idx, row in pg_existing_gens.iterrows():
    if row.operating_year >= pg_existing_gens.model_year.unique()[0]:
        pg_existing_gens.loc[idx,'operating_year'] = 2000


In [8]:
pg_existing_gens

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,...,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year,lifetime_yrs,end_of_life,Storage_Duration,model_year
0,BASN,Biomass,1,0.941,0.0,1.138,15.348,157220.000,5.730,1.650,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,...,0.05,0.05,NaN,NaN,NaN,2000,100,2100,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
4,BASN,Conventional Steam Coal,2,455.500,0.0,70.000,10.335,61341.714,1.879,0.358,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,SRCE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2396,SRSE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2397,SRSG,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2398,TRE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050


In [9]:
pg_existing_gens = pg_existing_gens[pg_existing_gens['Existing_Cap_MW'] >= MIN_CAP_THRESHOLD]
pg_existing_gens

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,...,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year,lifetime_yrs,end_of_life,Storage_Duration,model_year
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,...,0.05,0.05,NaN,NaN,NaN,2000,100,2100,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
4,BASN,Conventional Steam Coal,2,455.500,0.0,70.000,10.335,61341.714,1.879,0.358,...,0.06,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
5,BASN,Geothermal,1,9.551,0.0,5.259,8.843,209101.381,0.000,0.000,...,0.05,0.04,NaN,NaN,NaN,2000,100,2100,NaN,2027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,SRCE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2396,SRSE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2397,SRSG,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050
2398,TRE,distributed_generation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,NaN,NaN,2000,100,2100,NaN,2050


## Temporary Stuff

### Remove planning reserve margin

In [10]:
DEFAULT_VINTAGE = 2000

pg_capres = pg_capres.head(0)
pg_capres

,Network_zones,CapRes_1,CapRes_2,CapRes_3,CapRes_4,CapRes_5,CapRes_6,CapRes_7,CapRes_8,CapRes_9,CapRes_10


### Remove Small Hydroelectric from resource options

In [11]:
pg_new_gens = pg_new_gens.drop(pg_new_gens[pg_new_gens.technology == 'Small Hydroelectric'].index)

In [12]:
# Need to make some small edits to the pg_time dataframe, namely:
#   - t_day starts at 0, but should start at 1.
#   - same for timestep column
#   - these then need to be cast as strings.
pg_time["t_day"] += 1
pg_time["timestep"] += 1
pg_time = pg_time.astype({'t_day': 'str'}) 


In [13]:
pg_all_gens['tech_index'] = pg_all_gens.index


In [14]:
DEFAULT_VINTAGE = 2000


In [15]:
if not pg_existing_gens.empty:
    pg_existing_gens['operating_year'] = pg_existing_gens['operating_year'].fillna(DEFAULT_VINTAGE)
    pg_existing_gens = pg_existing_gens.astype({'operating_year':'int'})

if not pg_new_gens.empty:
    pg_new_gens['operating_year'] = pg_new_gens['operating_year'].fillna(DEFAULT_VINTAGE)
    pg_new_gens = pg_new_gens.astype({'operating_year':'int'})

pg_all_gens['operating_year'] = pg_all_gens['operating_year'].fillna(DEFAULT_VINTAGE)
pg_all_gens = pg_all_gens.astype({'operating_year':'int'})
    

/tmp/ipykernel_10400/1152768533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pg_existing_gens['operating_year'] = pg_existing_gens['operating_year'].fillna(DEFAULT_VINTAGE)


In [16]:
distributed_indices = pg_all_gens[pg_all_gens['technology'].str.contains("distributed")].index
pg_all_gens.loc[distributed_indices, "lifetime_yrs"] = 100
pg_all_gens.loc[distributed_indices, "end_of_life"] = 2100


### TEMPORARY: Remove existing technologies that have already 'retired'.

This will need to be revisted with either:
- updated lifetimes or vintage to prevent them from retiring before the model periods begin, or
- endogenous retirements are included in Temoa

In [17]:
# time_periods
planning_periods = list(pg_new_gens.operating_year.unique())
planning_periods.sort()

#pg_all_gens = pg_all_gens[pg_all_gens['end_of_life'] > planning_periods[0]]
#pg_new_gens = pg_new_gens[pg_new_gens['end_of_life'] > planning_periods[0]]
pg_existing_gens = pg_existing_gens[pg_existing_gens['end_of_life'] > planning_periods[0]]


In [18]:
if not pg_existing_gens.empty:
    pg_existing_gens['Existing_Cap_MW'] = pg_existing_gens['Existing_Cap_MW'].fillna(5) # Replace NaN with 5 MW

In [19]:
if not pg_existing_gens.empty:
    indices = pg_existing_gens['STOR'] == 1
    pg_existing_gens.loc[indices, 'Storage_Duration'] = pg_existing_gens[indices].Existing_Cap_MWh / pg_existing_gens[indices].Existing_Cap_MW
    pg_existing_gens = pg_existing_gens[pg_existing_gens['end_of_life'] > planning_periods[0]]


### TEMPORARY: If vintage of existing tech is in planning_years, switch it to default.


In [20]:
#indices = pg_existing_gens['operating_year'] >= current_period
#pg_existing_gens.loc[indices, 'operating_year'] = current_period - 1

### Remove flexible demand technologies for now

In [21]:
pg_all_gens = pg_all_gens[pg_all_gens.Flexible_Demand_Energy_Eff != 1]
if not pg_existing_gens.empty:
    pg_existing_gens = pg_existing_gens[pg_existing_gens.Flexible_Demand_Energy_Eff != 1]
pg_new_gens = pg_new_gens[pg_new_gens.Flexible_Demand_Energy_Eff != 1]


In [22]:
new_list = list(pg_new_gens.Resource)
new_set = set(pg_new_gens.Resource)
all_list = list(pg_all_gens.Resource)
all_set = set(pg_all_gens.Resource)

### Only use first model year for existing gens

In [23]:
if not pg_existing_gens.empty:
    pg_existing_gens['region+resource'] = pg_existing_gens[['region', 'Resource']].agg('-'.join, axis=1)
    pg_existing_gens = pg_existing_gens[pg_existing_gens.model_year == pg_existing_gens.model_year.min()]


## Set Creation

### Time

In [24]:
# time_periods
planning_periods = list(pg_new_gens.operating_year.unique())
planning_periods.sort()

In [25]:
if not pg_existing_gens.empty:
    vintages = list(pg_new_gens.operating_year.unique()) + list(pg_existing_gens.operating_year.unique())
else:
    vintages = list(pg_new_gens.operating_year.unique())
if DEFAULT_VINTAGE not in vintages:
    vintages.append(DEFAULT_VINTAGE)
vintages.sort()

In [26]:
# Temoa: time period labels
time_periods_dict = dict()
for v in vintages:
    time_periods_dict[v] = 'e'
for p in planning_periods:
    time_periods_dict[p] = 'f'
# add a final year to the time periods to determine length of final period.
if len(planning_periods) == 1:
    final_period_length = 1
else:
    final_period_length = planning_periods[-1] - planning_periods[-2]
time_periods_dict[p+final_period_length] = 'f'

temoa_time_periods = pd.DataFrame({'t_periods':time_periods_dict.keys(), 'flag':time_periods_dict.values()})

temoa_tables_dict['time_periods'] = temoa_time_periods
temoa_time_periods

,t_periods,flag
0,2000,e
1,2027,f
2,2030,f
3,2035,f
4,2040,f
5,2045,f
6,2050,f
7,2055,f


In [27]:
# Temoa: time_seasons_per_period
ts_per_p = {'periods':[],
                'season_name':[]}
for y in pg_time.year.unique():
    _df = pg_time[pg_time.year == y]
    _ts = list(_df.t_season.unique())
    ts_per_p['periods'] += [y] * len(_ts)
    ts_per_p['season_name'] += _ts

In [28]:
pg_time

,year,t_season,t_day,timestep,dayweight,SegFrac
0,2027,p12,1,1,168.0,0.000114
1,2027,p12,2,2,168.0,0.000114
2,2027,p12,3,3,168.0,0.000114
3,2027,p12,4,4,168.0,0.000114
4,2027,p12,5,5,168.0,0.000114
...,...,...,...,...,...,...
20155,2050,p1,164,3356,168.0,0.000114
20156,2050,p1,165,3357,168.0,0.000114
20157,2050,p1,166,3358,168.0,0.000114
20158,2050,p1,167,3359,168.0,0.000114


In [29]:
# Temoa: time_season and time_of_day
n_timesteps_per_day = 24
n_rep_periods = len(pg_time[pg_time.year == pg_time.year.unique()[0]].t_season.unique())
n_timesteps = pg_time.timestep.max()
n_days_per_rep_period = int(n_timesteps / n_timesteps_per_day / n_rep_periods)

In [30]:
temoa_time_of_day = {'t_day':[str(x+1) for x in range(n_days_per_rep_period*n_timesteps_per_day)]}
temoa_time_season = {'t_season':list(set(ts_per_p['season_name']))}

temoa_tables_dict['time_of_day'] = temoa_time_of_day
temoa_tables_dict['time_season'] = temoa_time_season
temoa_tables_dict['time_seasons_per_period'] = pd.DataFrame(ts_per_p)


### Regions

In [31]:
regions = list(pg_new_gens.region.unique())
regions.sort()

# for tx_path in pg_tx.transmission_path_name.unique():
#     r1, r2 = tx_path.split('_to_')
#     regions.append(r1 + '-' + r2)
#     regions.append(r2 + '-' + r1)
temoa_regions = pd.DataFrame({'regions':regions, 'region_note':['' for i in range(len(regions))]})

temoa_tables_dict['regions'] = temoa_regions

### Technologies

In [32]:
techs_new = list(pg_new_gens.Resource.unique())
techs_new.sort()
techs_exist = list(pg_existing_gens.Resource.unique())
techs_exist.sort()
all_techs = list(set(techs_new + techs_exist))
all_techs.append('transmission')

# Need to account for import technologies:
fuels = set(pg_existing_gens.Fuel).union(set(pg_new_gens.Fuel))
fuels.remove('None')
all_techs += ['import_' + _f for _f in fuels]

# Need to make a dummy technology for water into conventional hydro
for r in pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric'].Resource.unique():
    all_techs += ['water_import_' + str(r)]

# Need to add T&D tech
all_techs.append('elec_distribution')
all_techs.sort()
# For convenience, we create a dataframe that consists of all new resource options _not_ in the first
# planning year. 
# We do this because by convention the all_gens dataframe consists of all existing gens plus all
# gens in the first planning year. 
techs_storage = set(
    list(pg_new_gens[pg_new_gens.STOR == 1].Resource.unique()) +
    list(pg_existing_gens[pg_existing_gens.STOR == 1].Resource.unique()) +
    list(pg_new_gens[pg_new_gens.technology == 'Conventional Hydroelectric'].Resource.unique()) + 
    list(pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric'].Resource.unique())
)

In [33]:
tech=[]
flag=[]
sector=[]
tech_desc=[]
tech_category=[]

for t in all_techs:
    tech.append(t)
    flag.append('ps' if t in techs_storage else 'p')
    sector.append('electricity')
    tech_desc.append('')
    tech_category.append('')
    

temoa_technologies = pd.DataFrame({'tech':tech,
                                   'flag':flag,
                                   'sector':sector,
                                   'tech_desc':tech_desc,
                                   'tech_category':tech_category})
temoa_tables_dict['technologies'] = temoa_technologies

In [34]:
# tech curtail

# techs_curtail = list(set(
#     list(pg_new_gens[(pg_new_gens.VRE == 1) | (pg_new_gens.HYDRO == 1) | (pg_new_gens.MUST_RUN == 1)].Resource.unique()) + 
#     list(pg_existing_gens[(pg_existing_gens.VRE == 1) | (pg_existing_gens.HYDRO == 1) | (pg_existing_gens.MUST_RUN == 1)].Resource.unique())))
techs_curtail = list(set(
    list(pg_new_gens[(pg_new_gens.VRE == 1) | (pg_new_gens.MUST_RUN == 1)].Resource.unique()) + 
    list(pg_existing_gens[(pg_existing_gens.VRE == 1) | (pg_existing_gens.MUST_RUN == 1)].Resource.unique())))

techs_curtail.sort()
temoa_techs_curtail = pd.DataFrame({'tech':list(techs_curtail), 'notes':['']*len(techs_curtail)})
temoa_tables_dict['tech_curtailment'] = temoa_techs_curtail

In [35]:
# tech_exchange
temoa_tables_dict['tech_exchange'] = pd.DataFrame({'tech':'transmission',
                  'notes':' '}, index=[1])

In [36]:
# tech_ramping
if 'ramp' in scenario:
    techs_ramping = list(set(
        list(pg_new_gens[(pg_new_gens.Ramp_Up_Percentage < 1) | (pg_new_gens.Ramp_Up_Percentage < 1)].Resource.unique()) + 
        list(pg_all_gens[(pg_all_gens.Ramp_Up_Percentage < 1) | (pg_all_gens.Ramp_Up_Percentage < 1)].Resource.unique())))
    temoa_tables_dict['tech_ramping'] = {'tech':techs_ramping}

In [37]:
# tech_reserve

#CapResCols = [x for x in pg_all_gens.columns if 'CapRes' in x]
#pg_all_gens['CapRes_Total'] = pg_all_gens[CapResCols].sum(axis=1)
#cap_res_techs = pg_all_gens[pg_all_gens['CapRes_Total'] > 0].Resource.unique()

# The above code gets the list of all techs with CapRes > 0.
# But in Temoa we actually want all generators, so we use the below code instead:
gen_techs = [x for x in all_techs if 'import' not in x]
cap_res_techs = pd.DataFrame({'tech':gen_techs, 'notes':['']*len(gen_techs)})
temoa_tables_dict['tech_reserve'] = cap_res_techs

In [38]:

# # Define the column names
# columns = ["region", "group_name", "tech", "notes"]

# # Initialize an empty DataFrame with these columns
# temoa_tables_dict['tech_groups'] = pd.DataFrame(columns=columns)

### Commodities

In [39]:
fuels_new = list(pg_new_gens.Fuel.unique())
fuels_new = ['ethos' if x in ['None','none'] else x for x in fuels_new]
fuels_new.sort()
fuels_exist = list(pg_existing_gens.Fuel.unique())
fuels_exist = ['ethos' if x in ['None','none'] else x for x in fuels_exist]
if 'Conventional Hydroelectric' in pg_existing_gens.technology.unique():
    fuels_exist += ['water_gwh']
fuels_exist.sort()
all_fuels = list(set(fuels_new + fuels_exist))
all_fuels.sort()


emissions_comms = ['CO2'] # Will need to update this in the future.
electricity_comms = ['electricity'] # Will need to update this in the future.
demand_comms = ['DEMAND_ELC'] # Will need to update this in the future.


In [40]:
comm_name = []
flags = []
comm_desc = []
for flag, comms in zip(['p','e','d'],[all_fuels + electricity_comms, emissions_comms, demand_comms]):
    for comm in comms:
        comm_name.append(comm)
        flags.append(flag)
        comm_desc.append('')

temoa_commodities = pd.DataFrame({'comm_name':comm_name,
                                   'flag':flags,
                                   'comm_desc':comm_desc})
temoa_tables_dict['commodities'] = temoa_commodities

### Groups

In [41]:
temoa_tables_dict['groups'] = pd.DataFrame(columns=["group_name", "notes"])


## Parameter & Constraint Creation

### Efficiency

In [42]:
regions = []
input_comm = []
tech = []
vintage = []
output_comm = []
efficiency = []
eff_notes = []

for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        regions.append(row.region)
        in_comm = row.Fuel
        if in_comm in ['None', 'none']:
            in_comm = 'ethos'
        if row.STOR == 1: # the PG files have 'none' as the fuel type for batteries and pumped hydro. This should be electricity.
            in_comm = 'electricity'
        if row.technology == 'Conventional Hydroelectric':
            in_comm ='water_gwh'
        input_comm.append(in_comm)
        tech.append(row.Resource)
        vintage.append(row.operating_year)
        output_comm.append('electricity') # need to update this for certain techs (e.g., heaters and EVs?)
        if row.Resource in techs_storage:
            efficiency.append(row.Eff_Up * row.Eff_Down)
        elif row.Heat_Rate_MMBTU_per_MWh < 0.001:
            efficiency.append(1.0)
        elif type(row.profile) == str and len(row.profile) > 4:
            efficiency.append(1.0)
        else:
            efficiency.append(3.412/row.Heat_Rate_MMBTU_per_MWh)
        eff_notes.append('')
        
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    if max(abs(row.Line_Max_Flow_MW), abs(row.Line_Min_Flow_MW)) < 0.1:
        continue
    regions += [r1 + '-' + r2, r2 + '-' + r1]
    input_comm += ['electricity'] * 2
    tech += ['transmission'] * 2
    vintage += [DEFAULT_VINTAGE] * 2
    output_comm += ['electricity'] * 2
    efficiency += [1 - row.Line_Loss_Percentage] * 2
    eff_notes += ['']*2
    
    
for idx, row in pg_tx.iterrows():
    for p in planning_periods:
        r1, r2 = row.transmission_path_name.split('_to_')
        regions += [r1 + '-' + r2, r2 + '-' + r1]
        input_comm += ['electricity'] * 2
        tech += ['transmission'] * 2
        vintage += [p] * 2
        output_comm += ['electricity'] * 2
        efficiency += [1 - row.Line_Loss_Percentage] * 2
        eff_notes += ['']*2

            
            
# for r in pg_new_gens.region.unique():
#     for p in pg_new_gens.model_year.unique():
#         regions.append(r)
#         input_comm.append('electricity')
#         tech.append('elec_distribution')
#         vintage.append(p)
#         output_comm.append('DEMAND_ELC')
#         efficiency.append('1.0')
#         eff_notes.append('')
for r in pg_new_gens.region.unique():
        regions.append(r)
        input_comm.append('electricity')
        tech.append('elec_distribution')
        vintage.append(DEFAULT_VINTAGE)
        output_comm.append('DEMAND_ELC')
        efficiency.append('1.0')
        eff_notes.append('')
    
# Need to account for water flows for conventional hydro
_df = pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric']
for idx, row in _df.iterrows():
    regions.append(row.region)
    input_comm.append('ethos')
    tech.append('water_import_'+row.Resource)
    vintage.append(row.operating_year)
    output_comm.append('water_gwh')
    efficiency.append(1.0)
    eff_notes.append('Determined by capfacs')
    
temoa_efficiency = pd.DataFrame({
                    'regions':regions,
                    'input_comm':input_comm,
                    'tech':tech,
                    'vintage':vintage,
                    'output_comm':output_comm,
                    'efficiency':efficiency,
                    'eff_notes':eff_notes                
})



In [43]:
    
# Need to account for fuel imports
for r in temoa_efficiency.regions.unique():
    for fuel in temoa_efficiency[temoa_efficiency.regions == r].input_comm.unique():
        if fuel in ['ethos', 'electricity', 'water_gwh']:
            continue
        regions.append(r)
        input_comm.append('ethos')
        tech.append('import_' + fuel)
        vintage.append(DEFAULT_VINTAGE)
        output_comm.append(fuel)
        efficiency.append(1.0)
        eff_notes.append('')
        
temoa_efficiency = pd.DataFrame({
                    'regions':regions,
                    'input_comm':input_comm,
                    'tech':tech,
                    'vintage':vintage,
                    'output_comm':output_comm,
                    'efficiency':efficiency,
                    'eff_notes':eff_notes                
})      
temoa_tables_dict['Efficiency'] = temoa_efficiency
temoa_efficiency

,regions,input_comm,tech,vintage,output_comm,efficiency,eff_notes
0,BASN,waste_biomass,BASN_biomass_2,2000,electricity,0.360486,
1,BASN,water_gwh,BASN_conventional_hydroelectric_1,2000,electricity,1.0,
2,BASN,mountain_reference_coal,BASN_conventional_steam_coal_1,2000,electricity,0.322739,
3,BASN,mountain_reference_coal,BASN_conventional_steam_coal_2,2000,electricity,0.33014,
4,BASN,ethos,BASN_geothermal_1,2000,electricity,0.385842,
...,...,...,...,...,...,...,...
9151,TRE,ethos,import_hydrogen,2000,hydrogen,1.0,
9152,TREW,ethos,import_west_south_central_reference_naturalgas,2000,west_south_central_reference_naturalgas,1.0,
9153,TREW,ethos,import_west_south_central_reference_naturalgas...,2000,west_south_central_reference_naturalgas_ccs95,1.0,
9154,TREW,ethos,import_west_south_central_reference_uranium,2000,west_south_central_reference_uranium,1.0,


### Lifetime Tech

In [44]:
regions = []
tech = []
life = []
life_notes = []
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        regions.append(row.region)
        tech.append(row.Resource)
        life.append(row.lifetime_yrs)
        life_notes.append('')

# Tx
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    regions += [r1 + '-' + r2, r2 + '-' + r1]
    tech += ['transmission'] * 2
    life += [100] * 2
    life_notes += [''] * 2
    
# Imports
for idx, row in temoa_efficiency[temoa_efficiency['tech'].str.contains("import_")].iterrows():
    regions.append(row.regions)
    tech.append(row.tech)
    life.append(100)
    life_notes.append('')
    
for r in pg_new_gens.region.unique():
    regions.append(r)
    tech.append('elec_distribution')
    life.append(100)
    life_notes.append('')
    
_df = pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric']
for idx, row in _df.iterrows():
    regions.append(row.region)
    tech.append('water_import_'+row.Resource)
    life.append(100)
    life_notes.append('')
        
temoa_lifetimetech = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'life':life,
    'life_notes':life_notes
})
temoa_tables_dict['LifetimeTech'] = temoa_lifetimetech

temoa_lifetimetech

,regions,tech,life,life_notes
0,BASN,BASN_biomass_2,100,
1,BASN,BASN_conventional_hydroelectric_1,100,
2,BASN,BASN_conventional_steam_coal_1,100,
3,BASN,BASN_conventional_steam_coal_2,100,
4,BASN,BASN_geothermal_1,100,
...,...,...,...,...
8506,SRCA,water_import_SRCA_conventional_hydroelectric_1,100,
8507,SRCE,water_import_SRCE_conventional_hydroelectric_1,100,
8508,SRSE,water_import_SRSE_conventional_hydroelectric_1,100,
8509,SRSG,water_import_SRSG_conventional_hydroelectric_1,100,


### Capacity Credit -- This needs to be revisited.

In [45]:
regions = []
periods = []
tech = []
vintage = []
cf_tech = []
cf_tech_notes = []
cc_cols = [x for x in pg_existing_gens.columns if 'CapRes' in x]
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        v = row.operating_year
        lt = row.lifetime_yrs
        if 'wind' in row.Resource.lower():
            elcc = 0.1
        elif 'solar' in row.Resource.lower():
            elcc = 0.1
        elif 'pv' in row.Resource.lower():
            elcc = 0.1
        elif 'batt' in row.Resource.lower():
            elcc = 0.2
        else:
            elcc = max(row[cc_cols])
        for p in planning_periods:
            if p < v or p >= row.end_of_life:
                continue
            regions.append(row.region)
            periods.append(p)
            tech.append(row.Resource)
            vintage.append(v)
            cf_tech.append(elcc)
            cf_tech_notes.append('')
            
cc_cols = [x for x in pg_tx.columns if x[:6] == 'CapRes']
derate_cc_cols = [x for x in pg_tx.columns if x[:6] == 'Derate']
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    ccr1r2 = row[cc_cols].max() * row[derate_cc_cols].max()
    ccr2r1 = abs(row[cc_cols].min()) * row[derate_cc_cols].max()
    ## Just put them to zero for now... Needs to be updated.
    ccr1r2 = 0
    ccr2r1 = 0
    for p in planning_periods:
        regions += [r1 + '-' + r2, r2 + '-' + r1]
        periods += [p] * 2
        tech += ['transmission'] * 2
        vintage += [DEFAULT_VINTAGE] * 2
        cf_tech += [ccr1r2, ccr2r1]
        cf_tech_notes += [''] * 2
            
temoa_capacitycredit = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'vintage':vintage,
    'cf_tech':cf_tech,
    'cf_tech_notes':cf_tech_notes
})
temoa_tables_dict['CapacityCredit'] = temoa_capacitycredit

In [46]:
temoa_capacitycredit

,regions,periods,tech,vintage,cf_tech,cf_tech_notes
0,BASN,2027,BASN_biomass_2,2000,0.93,
1,BASN,2030,BASN_biomass_2,2000,0.93,
2,BASN,2035,BASN_biomass_2,2000,0.93,
3,BASN,2040,BASN_biomass_2,2000,0.93,
4,BASN,2045,BASN_biomass_2,2000,0.93,
...,...,...,...,...,...,...
29603,TRE-SRSG,2040,transmission,2000,0.00,
29604,SRSG-TRE,2045,transmission,2000,0.00,
29605,TRE-SRSG,2045,transmission,2000,0.00,
29606,SRSG-TRE,2050,transmission,2000,0.00,


In [47]:
df.Resource

1       BASN_naturalgas_hframe_cc_moderate_0
2       BASN_naturalgas_fframe_ct_moderate_0
3            BASN_nuclear_nuclear_moderate_0
4         BASN_hydrogen_fframe_ct_moderate_0
5         BASN_hydrogen_hframe_cc_moderate_0
                        ...                 
7765           TREW_battery_moderate_0_2hour
7766           TREW_battery_moderate_0_4hour
7767           TREW_battery_moderate_0_6hour
7768           TREW_battery_moderate_0_8hour
7769          TREW_battery_moderate_0_10hour
Name: Resource, Length: 7764, dtype: object


### CapacityFactorTech

In [48]:

regions = []
periods = []
season_name = []
time_of_day_name = []
tech = []
cf_tech = []
cf_tech_notes = []
n_timesteps = len(pg_time[pg_time.year == planning_periods[0]])

cf_eps = 0.01

for p in planning_periods:
    print("Year: ", p)
    pg_capfacs = pd.read_csv(results_directory + 'reduced_resource_' + str(p) + '.csv')
    df = pd.read_csv(results_directory + 'all_gens_' + str(p) + '.csv')
    for idx, row in df.iterrows():
        if row.MUST_RUN and 'distributed' not in row.technology: # must run techs are taken care of via a MinActivity constraint
            continue 
        if type(row.profile) != str: 
            continue
        if len(str(row.profile)) < 10: # some entries are "0", which makes things difficult... skip these
            continue
       # if row.Resource not in pg_existing_gens.Resource.unique():
       #     continue
        if row.Resource not in pg_existing_gens.Resource.unique(): # resource may have been removed from existing gens due to small capacity
            if row.Resource not in pg_new_gens.Resource.unique():
                continue
        v = row.operating_year
        lt = row.lifetime_yrs
        
        if p < v or p >= row.end_of_life:
            continue
        for s in pg_time[pg_time.year == p].t_season.unique():
            season_name += [s] * len(temoa_time_of_day['t_day'])
            time_of_day_name += temoa_time_of_day['t_day']
        regions += [row.region] * n_timesteps
        periods += [p] * n_timesteps
        if row.technology == 'Conventional Hydroelectric':
            tech += ['water_import_'+row.Resource] * n_timesteps
        else:
            tech += [row.Resource] * n_timesteps
        _cfs = np.asarray(pg_capfacs.loc[:, str(row.Resource)])
        small_val_flags = _cfs < cf_eps
        _cfs[small_val_flags] = 0
        cf_tech += list(_cfs)
        if len(list(pg_capfacs.loc[:, str(row.Resource)])) != n_timesteps:
            print(row.region, p, row.Resource, len(list(pg_capfacs.loc[:, str(row.Resource)])))
        cf_tech_notes += [''] * n_timesteps
    if '52' in scenario:
        break
if '52' in scenario:

    temoa_capacityfactortech = pd.DataFrame({
        'regions':regions,
        'season_name':season_name,
        'time_of_day_name':time_of_day_name,
        'tech':tech,
        'cf_tech':cf_tech,
        'cf_tech_notes':cf_tech_notes,
    })
else:
    temoa_capacityfactortech = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'season_name':season_name,
    'time_of_day_name':time_of_day_name,
    'tech':tech,
    'cf_tech':cf_tech,
    'cf_tech_notes':cf_tech_notes,
})


temoa_capacityfactortech.cf_tech = temoa_capacityfactortech.cf_tech.clip(0, 1)
temoa_tables_dict['CapacityFactorTech'] = temoa_capacityfactortech


Year:  2027
Year:  2030
Year:  2035
Year:  2040
Year:  2045
Year:  2050


In [49]:
# df = temoa_capacityfactortech
# results = df.groupby(['regions','periods','tech']).mean()
# results.to_csv('Annual_CapacityFactors.csv')


In [50]:
# # More memory efficient implementation...

# import pandas as pd
# import numpy as np

# # Define the total number of rows
# total_rows = 60242912

# # Preallocate memory
# regions = np.empty(total_rows, dtype='object')
# periods = np.empty(total_rows, dtype='int32')
# season_name = np.empty(total_rows, dtype='object')
# time_of_day_name = np.empty(total_rows, dtype='int32')
# tech = np.empty(total_rows, dtype='object')
# cf_tech = np.empty(total_rows, dtype='float32')
# cf_tech_notes = np.empty(total_rows, dtype='object')

# index = 0
# cf_eps = 0.01
# for p in planning_periods:
#     pg_capfacs = pd.read_csv(results_directory + 'reduced_resource_' + str(p) + '.csv')
#     df = pd.read_csv(results_directory + 'all_gens_' + str(p) + '.csv')
#     for idx, row in df.iterrows():
#         if row.MUST_RUN and 'distributed' not in row.technology:
#             continue
#         if not isinstance(row.profile, str):
#             continue
#         if len(row.profile) < 10:
#             continue
#         if row.Resource not in pg_existing_gens.Resource.unique() and row.Resource not in pg_new_gens.Resource.unique():
#             continue
#         if p < row.operating_year or p >= row.end_of_life:
#             continue
        
#         for s in pg_time[pg_time.year == p].t_season.unique():
#             seasons = [s] * len(temoa_time_of_day['t_day'])
#             times_of_day = temoa_time_of_day['t_day']
            
#             end_index = index + len(seasons)
#             season_name[index:end_index] = seasons
#             time_of_day_name[index:end_index] = times_of_day
        
#         end_index = index + n_timesteps
#         regions[index:end_index] = row.region
#         periods[index:end_index] = p
        
#         if row.technology == 'Conventional Hydroelectric':
#             tech[index:end_index] = 'water_import_' + row.Resource
#         else:
#             tech[index:end_index] = row.Resource
        
#         _cfs = np.asarray(pg_capfacs.loc[:, str(row.Resource)])
#         small_val_flags = _cfs < cf_eps
#         _cfs[small_val_flags] = 0
#         cf_tech[index:end_index] = _cfs
#         cf_tech_notes[index:end_index] = ''
        
#         index = end_index
        
# # Trim the arrays to the actual used size
# regions = regions[:index]
# periods = periods[:index]
# season_name = season_name[:index]
# time_of_day_name = time_of_day_name[:index]
# tech = tech[:index]
# cf_tech = cf_tech[:index]
# cf_tech_notes = cf_tech_notes[:index]


# # Create DataFrame
# temoa_capacityfactortech = pd.DataFrame({
#     'regions': pd.Categorical(regions),
#     'periods': pd.Categorical(periods),
#     'season_name': pd.Categorical(season_name),
#     'time_of_day_name': time_of_day_name,
#     'tech': pd.Categorical(tech),
#     'cf_tech': cf_tech,
#     'cf_tech_notes': cf_tech_notes,
# })

# temoa_capacityfactortech.cf_tech = temoa_capacityfactortech.cf_tech.clip(0, 1)
# temoa_tables_dict['CapacityFactorTech'] = temoa_capacityfactortech


In [51]:
temoa_capacityfactortech

,regions,periods,season_name,time_of_day_name,tech,cf_tech,cf_tech_notes
0,BASN,2027,p12,1,water_import_BASN_conventional_hydroelectric_1,0.475405,
1,BASN,2027,p12,2,water_import_BASN_conventional_hydroelectric_1,0.475405,
2,BASN,2027,p12,3,water_import_BASN_conventional_hydroelectric_1,0.475405,
3,BASN,2027,p12,4,water_import_BASN_conventional_hydroelectric_1,0.475405,
4,BASN,2027,p12,5,water_import_BASN_conventional_hydroelectric_1,0.475405,
...,...,...,...,...,...,...,...
22223035,TREW,2050,p1,164,TREW_utilitypv_class1_moderate_9_anyQual_1,0.000000,
22223036,TREW,2050,p1,165,TREW_utilitypv_class1_moderate_9_anyQual_1,0.000000,
22223037,TREW,2050,p1,166,TREW_utilitypv_class1_moderate_9_anyQual_1,0.000000,
22223038,TREW,2050,p1,167,TREW_utilitypv_class1_moderate_9_anyQual_1,0.000000,


### CapacityToActivity

In [52]:
regions = []
tech = []
c2a_tech = []
c2a_tech_notes = []
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        regions.append(row.region)
        tech.append(row.Resource)
        c2a_tech.append(8760)
        c2a_tech_notes.append('')

for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    regions += [r1 + '-' + r2, r2 + '-' + r1]
    tech += ['transmission'] * 2
    c2a_tech += [8760] * 2
    c2a_tech_notes += [''] * 2        
        
for r in pg_new_gens.region.unique():
    regions.append(r)
    tech.append('elec_distribution')
    c2a_tech.append(8760)
    c2a_tech_notes.append('')
    
    
# Need to account for fuel imports
for r in pg_new_gens.region.unique():
    for fuel in temoa_efficiency[temoa_efficiency.regions == r].input_comm.unique():
        if fuel in ['ethos', 'electricity', 'water_gwh']:
            continue
        regions.append(r)
        tech.append('import_' + fuel)
        c2a_tech.append(8760)
        c2a_tech_notes.append('')
        
_df = pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric']
for idx, row in _df.iterrows():
    regions.append(row.region)
    tech.append('water_import_'+row.Resource)
    c2a_tech.append(8760)
    c2a_tech_notes.append('')
    
temoa_capacitytoactivity = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'c2a':c2a_tech,
    'c2a_tech_notes':c2a_tech_notes
})
temoa_tables_dict['CapacityToActivity'] = temoa_capacitytoactivity


In [53]:
temoa_tables_dict['CapacityToActivity'][temoa_tables_dict['CapacityToActivity'].tech == 'SPPC_utilitypv_class1_moderate_9_anyQual_1']

,regions,tech,c2a,c2a_tech_notes
1176,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,
2470,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,
3764,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,
5058,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,
6352,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,
7646,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760,


### CostFixed





In [54]:
regions = []
periods = []
tech = []
vintage = []
cf_tech = []
cf_tech_units = []
cf_tech_notes = []
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        v = row.operating_year
        lt = row.lifetime_yrs
        if math.isnan(row['Fixed_OM_Cost_per_MWyr']):
            continue
        for p in planning_periods:
            if p < v or p >= row.end_of_life:
                continue
            regions.append(row.region)
            periods.append(p)
            tech.append(row.Resource)
            vintage.append(v)
            _cf = row['Fixed_OM_Cost_per_MWyr'] + row['co2_o_m_mw'] # $/MW 
            if row.STOR:
                if not math.isnan(row['Fixed_OM_Cost_per_MWhyr']):
                    _cf +=  row['Fixed_OM_Cost_per_MWhyr'] * row['Storage_Duration']
            cf_tech.append(_cf)
            cf_tech_units.append('$/MWyr')
            cf_tech_notes.append('')
            
# Assumption: Tx has no fixed costs. 

temoa_costfixed = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'vintage':vintage,
    'cost_fixed':cf_tech,
    'cost_fixed_units':cf_tech_units,
    'cost_fixed_notes':cf_tech_notes
})
temoa_tables_dict['CostFixed'] = temoa_costfixed

### CostInvest

In [55]:
regions = []
tech = []
vintage = []
ci = []
ci_units = []
ci_notes = []
for df in [pg_new_gens]:
    for idx, row in df.iterrows():
        regions.append(row.region)
        tech.append(row.Resource)
        vintage.append(row.operating_year)
        #_ci = row['Total Capex'] / 1000 #(row['capex_mw'] + row['interconnect_capex_mw'] + row['spur_capex']) / 1000 # $/MW to M$/GW
#         try:
#             _ci = (row['Total Capex'] + row['spur_capex'] + row['co2_pipeline_capex_mw'])  # $/MW 
#         except:
#             _ci = (row['Total Capex'] + row['spur_capex'])  # $/MW 
        _ci = (row['capex_mw'] * row['regional_cost_multiplier'] + \
               row['spur_capex'] + \
               row['co2_pipeline_capex_mw'] + \
               row['interconnect_capex_mw'])

        if row.STOR:
            _ci += row['capex_mwh']  * row['Storage_Duration'] * row['regional_cost_multiplier']
        ci.append(_ci) # $/MW to M$/GW
        ci_units.append('$/MW')
        ci_notes.append('')
        
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    for p in planning_periods:
        regions += [r1 + '-' + r2, r2 + '-' + r1]
        tech += ['transmission'] * 2
        vintage += [p] * 2
        ci += [row.Line_Reinforcement_Cost_per_MW / 2 ] * 2 # Investment costs for Tx in Temoa are halved
                                                                  # because it is paid for in both directions.
        ci_notes += [''] * 2        
        ci_units += ['$/MW'] * 2
            

    
temoa_costinvest = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'vintage':vintage,
    'cost_invest':ci,
    'cost_invest_units':ci_units,
    'cost_invest_notes':ci_notes
})
temoa_tables_dict['CostInvest'] = temoa_costinvest

### CostVariable

In [56]:
regions = []
periods = []
tech = []
vintage = []
cv_tech = []
cv_tech_units = []
cv_tech_notes = []
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        v = row.operating_year
        lt = row.lifetime_yrs
        if math.isnan(row['Var_OM_Cost_per_MWh']):
            continue
        for p in planning_periods:
            if p < v or p >= row.end_of_life:
                continue
            regions.append(row.region)
            periods.append(p)
            tech.append(row.Resource)
            vintage.append(v)
            #cv_tech.append(row['Var_OM_Cost_per_MWh']) # $/MWh to M$/TWh
#             try:
#                 cv_tech.append((row['Var_OM_Cost_per_MWh']+row['co2_cost_mwh'])) # $/MWh to M$/GWh
#             except:
#                 cv_tech.append(row['Var_OM_Cost_per_MWh']) # $/MWh 
            cv_tech.append((row['Var_OM_Cost_per_MWh']+row['co2_cost_mwh']))
            cv_tech_units.append('$/MWh')
            cv_tech_notes.append('')
            
for idx, row in temoa_efficiency[temoa_efficiency['tech'].str.contains("import_")].iterrows():
    if 'water' in row.tech: # skip this for the dummy water_import tech.
        continue
    for p in planning_periods:
        regions.append(row.regions)
        periods.append(p)
        tech.append(row.tech)
        vintage.append(row.vintage)
        #cv = pg_fuel_price[pg_fuel_price.year == p][row.output_comm].mean() / 0.29307 # $/MMBtu to M$/GWh
        cv = pg_fuel_price[pg_fuel_price.year == p][row.output_comm].mean() / 293.07 * 1000 # $/MMBtu to $/MWh
        cv_tech.append(cv)
        cv_tech_units.append('$/MWh')
        cv_tech_notes.append('')

temoa_costvar = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'vintage':vintage,
    'cost_variable':cv_tech,
    'cost_variable_units':cv_tech_units,
    'cost_variable_notes':cv_tech_notes
})
temoa_tables_dict['CostVariable'] = temoa_costvar


In [57]:
pg_demand

,BASN,CANO,CASO,FRCC,ISNE,MISC,MISE,MISS,MISW,NWPP,...,SPPC,SPPN,SPPS,SRCA,SRCE,SRSE,SRSG,TRE,TREW,year
0,10262.0,11684.0,18026.0,17934.0,11354.0,14823.0,7747.0,15092.0,17863.0,22457.0,...,6011.0,3726.0,9931.0,15583.0,16716.0,18981.0,12014.0,35218.0,1565.0,2027
1,9786.0,11179.0,17246.0,16849.0,10619.0,14131.0,7455.0,14398.0,17035.0,21115.0,...,5703.0,3548.0,9539.0,15053.0,15967.0,17992.0,11310.0,33217.0,1476.0,2027
2,9328.0,10471.0,16161.0,16228.0,10304.0,13660.0,7286.0,13891.0,16544.0,19965.0,...,5491.0,3433.0,9208.0,14798.0,15444.0,17393.0,10909.0,31821.0,1414.0,2027
3,9002.0,9843.0,15196.0,16059.0,10116.0,13364.0,7201.0,13599.0,16385.0,19273.0,...,5393.0,3396.0,9099.0,14926.0,15270.0,17117.0,10539.0,30983.0,1376.0,2027
4,8803.0,9559.0,14762.0,16429.0,10011.0,13272.0,7238.0,13431.0,16384.0,19084.0,...,5370.0,3390.0,8976.0,15212.0,15350.0,17109.0,10401.0,30479.0,1354.0,2027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20155,32008.0,24413.0,37761.0,50746.0,33647.0,43166.0,22133.0,37281.0,55213.0,61053.0,...,17543.0,12742.0,27795.0,44184.0,45726.0,53023.0,32572.0,82072.0,3647.0,2050
20156,33000.0,26030.0,40264.0,47461.0,32427.0,42190.0,21240.0,36495.0,54040.0,61036.0,...,17348.0,12544.0,27523.0,41220.0,43983.0,50341.0,34280.0,81945.0,3642.0,2050
20157,34048.0,27847.0,43075.0,43374.0,31584.0,40332.0,20107.0,34688.0,50946.0,63726.0,...,16521.0,11907.0,26340.0,37985.0,41271.0,46815.0,34752.0,78158.0,3473.0,2050
20158,33181.0,27230.0,42123.0,39124.0,30767.0,38227.0,18434.0,32867.0,47980.0,62298.0,...,15626.0,11186.0,24961.0,34123.0,38070.0,42621.0,33731.0,73536.0,3268.0,2050


### Demand

In [58]:
regions = []
periods = []
demand_comm = []
demand = []
demand_units = []
demand_notes = []

# Note, PG outputs only the actual demand in each timestep.
# We need to scale this appropriately

for y in pg_demand.year.unique():
    _df = pg_demand[pg_demand.year == y]
    _sf = pg_time[pg_time.year == y].SegFrac.values
    for r in pg_new_gens.region.unique():
        regions.append(r)
        periods.append(y)
        demand_comm.append('DEMAND_ELC')
        _dem = (_df[r].values * _sf * 8760).sum()
        #demand.append(_dem/1000000) # TWh
        demand.append(_dem)   # GWh
        demand_units.append('MWh')
        demand_notes.append('')
        

temoa_demand = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'demand_comm':demand_comm,
    'demand':demand,
    'demand_units':demand_units,
    'demand_notes':demand_notes
})
temoa_tables_dict['Demand'] = temoa_demand
temoa_demand

,regions,periods,demand_comm,demand,demand_units,demand_notes
0,BASN,2027,DEMAND_ELC,1.059955e+08,MWh,
1,CANO,2027,DEMAND_ELC,1.033222e+08,MWh,
2,CASO,2027,DEMAND_ELC,1.594922e+08,MWh,
3,FRCC,2027,DEMAND_ELC,2.517537e+08,MWh,
4,ISNE,2027,DEMAND_ELC,1.301105e+08,MWh,
...,...,...,...,...,...,...
151,SRCE,2050,DEMAND_ELC,3.334336e+08,MWh,
152,SRSE,2050,DEMAND_ELC,3.994389e+08,MWh,
153,SRSG,2050,DEMAND_ELC,2.561549e+08,MWh,
154,TRE,2050,DEMAND_ELC,6.487250e+08,MWh,


In [59]:
temoa_demand[(temoa_demand.regions=='BASN') & (temoa_demand.periods == 2030)].demand.sum()

119289517.59613034

### DemandSpecificDistribution

In [60]:
regions = []
periods = []
season_name = []
time_of_day_name = []
demand_name = []
dds = []
dds_notes = []

for y in pg_demand.year.unique():
    _df = pg_demand[pg_demand.year == y]
    _dft = pg_time[pg_time.year == y]
    n_ts = len(_dft)
    for r in pg_new_gens.region.unique():
        regions += [r] * n_ts
        periods += [y] * n_ts
        season_name += list(_dft.t_season)
        time_of_day_name += list(_dft.t_day)
        demand_name += ['DEMAND_ELC'] * n_ts
        _dds = _df[r].values * _dft.SegFrac
        _dds /= sum(_dds)
        dds += list(_dds)
        dds_notes += [''] * n_ts

temoa_dsd = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'season_name':season_name,
    'time_of_day_name':time_of_day_name,
    'demand_name':demand_name,
    'dds':dds,
    'dds_notes':dds_notes
})


temoa_tables_dict['DemandSpecificDistribution'] = temoa_dsd


### DiscountRate

In [61]:
regions = []
tech = []
vintage = []
tech_rate = []
tech_rate_notes = []
for df in [pg_new_gens]:
    for idx, row in df.iterrows():
        regions.append(row.region)
        tech.append(row.Resource)
        vintage.append(row.operating_year)
        tech_rate.append(row['wacc_real'])
        tech_rate_notes.append('')
        
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    for p in planning_periods:
        regions += [r1 + '-' + r2, r2 + '-' + r1]
        tech += ['transmission'] * 2
        vintage += [p] * 2
#         tech_rate += [row.WACC] * 2
        tech_rate += [0.05] * 2
        tech_rate_notes += [''] * 2        

temoa_discountrate = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'vintage':vintage,
    'tech_rate':tech_rate,
    'tech_rate_notes':tech_rate_notes
})
temoa_tables_dict['DiscountRate'] = temoa_discountrate

### EmissionActivity

Note: Still need to think about CCS technologies.

In [62]:
regions = []
emis_comm = []
input_comm = []
tech = []
vintage = []
output_comm = []
emis_act = []
emis_act_units = []
emis_act_notes = []

   
for idx, row in temoa_efficiency[temoa_efficiency['tech'].str.contains("import_")].iterrows():
    if 'water' in row.tech: 
        continue
    ei = pg_fuel_co2[pg_fuel_co2.Fuel == row.output_comm].CO2_Intensity_tons_per_MMBtu.values[0]
    ei /= 0.29307 # t per MMBtu to kt per GWh
    if ei < 0.0001:
        continue
    regions.append(row.regions)
    emis_comm.append('CO2')
    input_comm.append(row.input_comm)
    tech.append(row.tech)
    vintage.append(row.vintage)
    output_comm.append(row.output_comm)
    #emis_act.append(ei*1000)  # TWh
    emis_act.append(ei) # GWh
    emis_act_units.append('t/MWh')
    emis_act_notes.append('')

temoa_emissionactivity = pd.DataFrame({
    'regions':regions,
    'emis_comm':emis_comm,
    'input_comm':input_comm,
    'tech':tech,
    'vintage':vintage,
    'output_comm':output_comm,
    'emis_act':emis_act,
    'emis_act_units':emis_act_units,
    'emis_act_notes':emis_act_notes,
})
temoa_tables_dict['EmissionActivity'] = temoa_emissionactivity
temoa_emissionactivity

,regions,emis_comm,input_comm,tech,vintage,output_comm,emis_act,emis_act_units,emis_act_notes
0,BASN,CO2,ethos,import_mountain_reference_coal,2000,mountain_reference_coal,0.325929,t/MWh,
1,BASN,CO2,ethos,import_mountain_reference_naturalgas,2000,mountain_reference_naturalgas,0.181049,t/MWh,
2,BASN,CO2,ethos,import_mountain_reference_distillate,2000,mountain_reference_distillate,0.249599,t/MWh,
3,CANO,CO2,ethos,import_pacific_reference_naturalgas,2000,pacific_reference_naturalgas,0.181049,t/MWh,
4,CANO,CO2,ethos,import_pacific_reference_distillate,2000,pacific_reference_distillate,0.249599,t/MWh,
...,...,...,...,...,...,...,...,...,...
84,TRE,CO2,ethos,import_west_south_central_reference_coal,2000,west_south_central_reference_coal,0.325929,t/MWh,
85,TRE,CO2,ethos,import_west_south_central_reference_naturalgas,2000,west_south_central_reference_naturalgas,0.181049,t/MWh,
86,TRE,CO2,ethos,import_west_south_central_reference_naturalgas...,2000,west_south_central_reference_naturalgas_ccs95,0.009042,t/MWh,
87,TREW,CO2,ethos,import_west_south_central_reference_naturalgas,2000,west_south_central_reference_naturalgas,0.181049,t/MWh,


### EmissionLimit


In [63]:
_df = pg_policies[(pg_policies.case_id == scenario) | (pg_policies.case_id == 'all')]
_df

,Unnamed: 0,case_id,year,region,ESR_1,ESR_2,ESR_3,ESR_4,ESR_5,ESR_6,...,ESR_11,ESR_12,ESR_13,ESR_14,ESR_15,ESR_16,CO_2_Cap_Zone_1,CO_2_Max_Mtons_1,copy_case_id,Note
0,0,all,2027,BASN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,873.0,NaN,halfway between 2023 (1560 MMT according to EP...
1,1,all,2027,CANO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2,all,2027,CASO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,3,all,2027,FRCC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,4,all,2027,ISNE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,151,all,2050,SRCE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
152,152,all,2050,SRSE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
153,153,all,2050,SRSG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
154,154,all,2050,TRE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [64]:
if _df.copy_case_id.iloc[0] in pg_policies.case_id.unique():
    _df = pg_policies[pg_policies.case_id == _df.copy_case_id.iloc[0]]

In [65]:
pg_policies

,Unnamed: 0,case_id,year,region,ESR_1,ESR_2,ESR_3,ESR_4,ESR_5,ESR_6,...,ESR_11,ESR_12,ESR_13,ESR_14,ESR_15,ESR_16,CO_2_Cap_Zone_1,CO_2_Max_Mtons_1,copy_case_id,Note
0,0,all,2027,BASN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,873.0,NaN,halfway between 2023 (1560 MMT according to EP...
1,1,all,2027,CANO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2,all,2027,CASO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,3,all,2027,FRCC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,4,all,2027,ISNE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,151,all,2050,SRCE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
152,152,all,2050,SRSE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
153,153,all,2050,SRSG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
154,154,all,2050,TRE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [66]:
co2_cap_zones_to_regions = dict()
co2_cap_cols = [col for col in _df.columns if 'CO_2_Cap_Zone' in col]
for col in co2_cap_cols:
    co2_cap_zones_to_regions[col] = list(_df[_df[col] == 1].region.unique())
co2_cap_zones_to_regions

{'CO_2_Cap_Zone_1': ['BASN',
  'CANO',
  'CASO',
  'FRCC',
  'ISNE',
  'MISC',
  'MISE',
  'MISS',
  'MISW',
  'NWPP',
  'NYCW',
  'NYUP',
  'PJMC',
  'PJMD',
  'PJME',
  'PJMW',
  'RMRG',
  'SPPC',
  'SPPN',
  'SPPS',
  'SRCA',
  'SRCE',
  'SRSE',
  'SRSG',
  'TRE',
  'TREW']}

In [67]:
regions = []
periods = []
emis_comm = []
emis_limit = []
emis_limit_units = []
emis_limit_notes = []
for zones in co2_cap_zones_to_regions:
    zone_num = zones.split('_')[-1]
    _regions = co2_cap_zones_to_regions[zones]
    if len(_regions) == len(pg_existing_gens.region.unique()):
        _regions = 'global'
    elif len(_regions) > 1:
        _regions = '+'.join(_regions)
    for y in _df.year.unique():
        if y not in planning_periods:
            continue
        regions.append(_regions)
        periods.append(y)
        emis_comm.append('CO2')
        _el = _df[_df.year == y]['CO_2_Max_Mtons_'+str(zone_num)].sum() * 1000000
        emis_limit.append(_el)
        emis_limit_units.append('t')
        emis_limit_notes.append('')
    
temoa_emissionlimit = pd.DataFrame({
        'regions':regions,
        'periods':periods,
        'emis_comm':emis_comm,
        'emis_limit':emis_limit,
        'emis_limit_units':emis_limit_units,
        'emis_limit_notes':emis_limit_notes,
    })
temoa_tables_dict['EmissionLimit'] = temoa_emissionlimit
temoa_emissionlimit

,regions,periods,emis_comm,emis_limit,emis_limit_units,emis_limit_notes
0,global,2027,CO2,873000000.0,t,
1,global,2030,CO2,186000000.0,t,
2,global,2035,CO2,130000000.0,t,
3,global,2040,CO2,86660000.0,t,
4,global,2045,CO2,43330000.0,t,
5,global,2050,CO2,0.0,t,


In [68]:
region_combinations = list(set([r for r in regions if '+' in r]))
temoa_regioncombinations = pd.DataFrame({'regions':region_combinations, 'region_note':['' for i in range(len(region_combinations))]})

temoa_tables_dict['region_combinations'] = temoa_regioncombinations

### ExistingCapacity
Note: For lines where the max flow given by powergenome is asymmetric, we assume the existing capacity is the larger of the two values. I'm currently waiting to hear back from Jesse J. to ensure our assumption is consistent with theirs in GenX.

In [69]:
pg_existing_gens.model_year.unique()

array([2027])

In [70]:
regions = []
tech = []
vintage = []
existing_cap = []
existing_cap_units = []
existing_cap_notes = []

for idx, row in pg_existing_gens.iterrows():
    regions.append(row.region)
    tech.append(row.Resource)
    vintage.append(row.operating_year)
    if math.isnan(row.Existing_Cap_MW): # Bug in powergenome. Revisit this
        print(row.Resource)
        existing_cap.append(0.01)
    else:
        existing_cap.append(row.Existing_Cap_MW)
    existing_cap_units.append('MW')
    existing_cap_notes.append('')
    
    
    
    
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    if max(abs(row.Line_Max_Flow_MW), abs(row.Line_Min_Flow_MW)) < 0.1:
        continue
    regions += [r1 + '-' + r2, r2 + '-' + r1]
    tech += ['transmission'] * 2
    vintage += [DEFAULT_VINTAGE] * 2
    existing_cap += [max(abs(row.Line_Max_Flow_MW), abs(row.Line_Min_Flow_MW))] * 2
    existing_cap_units += ['MW'] * 2
    existing_cap_notes += [''] * 2
    
    
# Need to account for fuel imports
# We cannot size these in the first planning periods, as we have in the past, because
# this won't work in a myopic setting.
        
for r in temoa_efficiency.regions.unique():
    for fuel in temoa_efficiency[temoa_efficiency.regions == r].input_comm.unique():
        if fuel in ['ethos', 'electricity', 'water_gwh']:
            continue
        regions.append(r)
        tech.append('import_' + fuel)
        vintage.append(DEFAULT_VINTAGE)
        peak_dem_gw = pg_demand[r].max()  # Find the peak load, convert to GW
        existing_cap.append(peak_dem_gw*3) # we just need to make sure there is more than enough import capacity for all periods
        existing_cap_units.append('MW')
        existing_cap_notes.append('')
        
# Need to account for water flows for conventional hydro
_df = pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric']
for idx, row in _df.iterrows():
    regions.append(row.region)
    tech.append('water_import_'+row.Resource)
    vintage.append(row.operating_year)
    if math.isnan(row.Existing_Cap_MW): # Bug in powergenome. Revisit this
        print(row.Resource)
        existing_cap.append(0.01)
    else:
        existing_cap.append(row.Existing_Cap_MW)
    existing_cap_units.append('MW')
    existing_cap_notes.append('')

        
for r in pg_new_gens.region.unique():
    regions.append(r)
    tech.append('elec_distribution')
    vintage.append(DEFAULT_VINTAGE)
    peak_dem_gw = pg_demand[r].max()  # Find the peak load, convert to GW
    existing_cap.append(peak_dem_gw*3) # we just need to make sure there is more than enough import capacity for all periods
    existing_cap_units.append('MW')
    existing_cap_notes.append('')
    
temoa_existingcapacity = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'vintage':vintage,
    'exist_cap':existing_cap,
    'exist_cap_units':existing_cap_units,
    'exist_cap_notes':existing_cap_notes,
})
temoa_tables_dict['ExistingCapacity'] = temoa_existingcapacity
temoa_existingcapacity

,regions,tech,vintage,exist_cap,exist_cap_units,exist_cap_notes
0,BASN,BASN_biomass_2,2000,16.776,MW,
1,BASN,BASN_conventional_hydroelectric_1,2000,2489.520,MW,
2,BASN,BASN_conventional_steam_coal_1,2000,1178.200,MW,
3,BASN,BASN_conventional_steam_coal_2,2000,1822.000,MW,
4,BASN,BASN_geothermal_1,2000,420.244,MW,
...,...,...,...,...,...,...
703,SRCE,elec_distribution,2000,197589.000,MW,
704,SRSE,elec_distribution,2000,252189.000,MW,
705,SRSG,elec_distribution,2000,164916.000,MW,
706,TRE,elec_distribution,2000,400929.000,MW,


### GlobalDiscountRate

In [71]:
temoa_tables_dict['GlobalDiscountRate'] = {'rate':[0.02]}

### GrowthRateMax

In [72]:
# From scenario_management.yml
# limitedtransmission:
#      yes:
#        tx_expansion_per_period: 0.5

### GrowthRateSeed

### LifetimeLoanTech

In [73]:
regions = []
tech = []
loan = []
loan_notes = []

for idx, row in pg_new_gens.iterrows():
    regions.append(row.region)
    tech.append(row.Resource)
    loan.append(row.cap_recovery_years)
    loan_notes.append('')
    
for idx, row in pg_tx.iterrows():
    r1, r2 = row.transmission_path_name.split('_to_')
    regions += [r1 + '-' + r2, r2 + '-' + r1]
    tech += ['transmission'] * 2
    loan += [row.investment_years] * 2
    loan_notes += ['']*2

    
temoa_loanlifetime = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'loan':loan,
    'loan_notes':loan_notes
})
temoa_tables_dict['LifetimeLoanTech'] = temoa_loanlifetime

### MaxActivity

### MaxCapacity

In [74]:
regions = []
periods = []
tech = []
maxcap = []
maxcap_units = []
maxcap_notes = []
primary_keys = set() # (r, p, t)

for idx, row in pg_new_gens.iterrows():
    _maxcap = row.Max_Cap_MW 
    if _maxcap < 0: # if unconstrained, entry is -1. 
        continue
    for y in pg_time.year.unique():
        if row.operating_year < y:
            continue
        _idx = (row.region, y, row.Resource)
        if _idx in primary_keys:
            continue
        primary_keys.add(_idx)
        regions.append(row.region)
        periods.append(y)
        tech.append(row.Resource)
        maxcap.append(_maxcap)
        maxcap_notes.append('')
        maxcap_units.append('MW')

# for idx, row in pg_tx.iterrows():
#     _maxcap = row.Line_Max_Reinforcement_MW / 1000 # GW
#     if _maxcap < 0: # if unconstrained, entry is -1
#         continue
#     _excap = max(abs(row.Line_Max_Flow_MW), abs(row.Line_Min_Flow_MW)) / 1000
#     _maxcap = max(_excap,_maxcap)
#     for y in pg_time.year.unique():
#         r1, r2 = row.transmission_path_name.split('_to_')
#         _idx = (r1 + '-' + r2, y, 'transmission')
#         if _idx in primary_keys:
#             continue
#         primary_keys.add(_idx)
#         regions += [r1 + '-' + r2, r2 + '-' + r1]
#         periods += [y] * 2
#         tech += ['transmission'] * 2
#         maxcap += [_maxcap] * 2
#         maxcap_units += ['GW'] * 2
#         maxcap_notes += [''] * 2

    
temoa_maxcapacity = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'maxcap':maxcap,
    'maxcap_units':maxcap_units,
    'maxcap_notes':maxcap_notes
})


for i,y in enumerate(pg_time.year.unique()):
    temoa_maxcapacity.loc[temoa_maxcapacity.periods == y, 'maxcap'] += i*10 # let an additional 10 MW per planning year.. for computational reasons
    
temoa_tables_dict['MaxCapacity'] = temoa_maxcapacity


### MaxNewCapacity

The max capacities included in the network/transmission file are reported as max capacity additions per planning period.


In [75]:
regions = []
periods = []
tech = []
maxcap = []
maxcap_units = []
maxcap_notes = []
primary_keys = set() # (r, p, t)

for idx, row in pg_tx.iterrows():
    _maxcap = row.Line_Max_Reinforcement_MW
    if _maxcap < 0: # if unconstrained, entry is -1
        continue
#     _excap = max(abs(row.Line_Max_Flow_MW), abs(row.Line_Min_Flow_MW)) / 1000
#     _maxcap = max(_excap,_maxcap)
    for y in pg_time.year.unique():
        r1, r2 = row.transmission_path_name.split('_to_')
        _idx = (r1 + '-' + r2, y, 'transmission')
        if _idx in primary_keys:
            continue
        primary_keys.add(_idx)
        regions += [r1 + '-' + r2, r2 + '-' + r1]
        periods += [y] * 2
        tech += ['transmission'] * 2
        maxcap += [_maxcap] * 2
        maxcap_units += ['MW'] * 2
        maxcap_notes += [''] * 2

    
temoa_maxnewcapacity = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'maxnewcap':maxcap,
    'maxnewcap_units':maxcap_units,
    'maxnewcap_notes':maxcap_notes
})

  
temoa_tables_dict['MaxNewCapacity'] = temoa_maxnewcapacity

### MaxResource

### MinActivity

Note: Temoa does not include the notion of "MUST RUN". Power genome has a MUST_RUN category for generators, and this means the generators must run at a 100% utilization factor (based on their adjusted capacity -- but we use the adjusted capacity as the nameplate capacity, so here we force them to run at 100%).

This will only work for existing techs, but this is the case. Will need to revisit should this change.


### MinCapacityFactor

Note: Temoa does not include the notion of "MUST RUN". Power genome has a MUST_RUN category for generators, and this means the generators must run at a 100% utilization factor (based on their adjusted capacity -- but we use the adjusted capacity as the nameplate capacity, so here we force them to run at 100%).


In [76]:
regions = []
periods = []
tech = []
mincf = []
mincf_notes = []
primary_kys = set() # (r, p, t)

for idx, row in pg_existing_gens[pg_existing_gens.MUST_RUN == 1].iterrows():
    if 'distributed' in row.technology:
        continue
    for y in pg_time.year.unique():
        if row.operating_year > y or y >= row.end_of_life:
            continue
        regions.append(row.region)
        periods.append(y)
        tech.append(row.Resource)
        mincf.append(0.97)
        mincf_notes.append('')
        
    
temoa_minannualcapacityfactor = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'mincf':mincf,
    'mincf_notes':mincf_notes
})
temoa_tables_dict['MinAnnualCapacityFactor'] = temoa_minannualcapacityfactor
temoa_minannualcapacityfactor

,regions,periods,tech,mincf,mincf_notes
0,BASN,2027,BASN_biomass_2,0.97,
1,BASN,2030,BASN_biomass_2,0.97,
2,BASN,2035,BASN_biomass_2,0.97,
3,BASN,2040,BASN_biomass_2,0.97,
4,BASN,2045,BASN_biomass_2,0.97,
...,...,...,...,...,...
295,TRE,2030,TRE_small_hydroelectric_1,0.97,
296,TRE,2035,TRE_small_hydroelectric_1,0.97,
297,TRE,2040,TRE_small_hydroelectric_1,0.97,
298,TRE,2045,TRE_small_hydroelectric_1,0.97,


### MinCapacity

### MinGenGroupTarget

### MinGenGroupWeight

### PlanningReserveMargine

In [77]:
regions = []
reserve_margin = []

cap_res_zones = pg_capres.columns[1:]
if not pg_capres.empty:
    for crz in cap_res_zones:
        _regions = []
        _capres = []
        for idx, row in pg_capres.iterrows():
            if row[crz] > 0:
                _regions.append(row.Network_zones)
                _capres.append(row[crz])
        regions.append('+'.join(map(str, _regions)) )
        reserve_margin.append(statistics.mean(_capres))

            
temoa_planningreservemargin = pd.DataFrame({
    'regions':regions,
    'reserve_margin':reserve_margin
})
temoa_tables_dict['PlanningReserveMargin'] = temoa_planningreservemargin

In [78]:
region_combinations = set([r for r in regions if '+' in r and r])
region_notes = [''] * len(region_combinations)
temoa_regioncombinations = pd.DataFrame({'regions':list(region_combinations), 'region_note':region_notes})

temoa_tables_dict['region_combinations'] = pd.concat([temoa_tables_dict['region_combinations'], temoa_regioncombinations])
temoa_tables_dict['region_combinations']

,regions,region_note


### RampDown

In [79]:
if 'ramp' in scenario:
    regions = []
    tech = []
    ramp = []

    for df in [pg_new_gens, pg_existing_gens]:
        if 'Ramp_Dn_Percentage' not in df.columns:
            continue
        for idx, row in df.iterrows():
            r = min(1,row.Ramp_Up_Percentage)
            if r >= 1:
                continue
            regions.append(row.region)
            tech.append(row.Resource)
            ramp.append(r)


    temoa_rampdown = pd.DataFrame({
        'regions':regions,
        'tech':tech,
        'ramp_down':ramp
    })
    temoa_tables_dict['RampDown'] = temoa_rampdown

### RampUp

In [80]:
if 'ramp' in scenario:
    regions = []
    tech = []
    ramp = []

    for df in [pg_new_gens, pg_existing_gens]:
        if 'Ramp_Up_Percentage' not in df.columns:
            continue
        for idx, row in df.iterrows():
            r = min(1,row.Ramp_Up_Percentage)
            if r >= 1:
                continue
            regions.append(row.region)
            tech.append(row.Resource)
            ramp.append(r)


    temoa_rampup = pd.DataFrame({
        'regions':regions,
        'tech':tech,
        'ramp_up':ramp
    })
    temoa_tables_dict['RampUp'] = temoa_rampup

### SegFrac

In [81]:
periods = []
season_name = []
time_of_day_name = []
segfrac = []
segfrac_notes = []

for y in pg_demand.year.unique():
    _dft = pg_time[pg_time.year == y]
    n_ts = len(_dft)
    periods += [y] * n_ts
    season_name += list(_dft.t_season)
    time_of_day_name += list(_dft.t_day)
    segfrac += list(_dft.SegFrac)
    segfrac_notes += [''] * n_ts
    

temoa_segfrac = pd.DataFrame({
    'periods':periods,
    'season_name':season_name,
    'time_of_day_name':time_of_day_name,
    'segfrac':segfrac,
    'segfrac_notes':segfrac_notes
})


temoa_tables_dict['SegFrac'] = temoa_segfrac

### StorageDuration
Note: Need to revisit this.

In [82]:
regions = []
tech = []
duration = []
duration_notes = []
for df in [pg_existing_gens, pg_new_gens]:
    for idx, row in df.iterrows():
        if row.STOR == 0:
            continue
        regions.append(row.region)
        tech.append(row.Resource)
        duration.append(row.Storage_Duration)
        duration_notes.append('')
        
# Need to account for water flows for conventional hydro
_df = pg_existing_gens[pg_existing_gens.technology == 'Conventional Hydroelectric']
assumed_energy_to_power_ratio = 1
for idx, row in _df.iterrows():
    regions.append(row.region)
    tech.append(row.Resource)
    duration.append(assumed_energy_to_power_ratio)
    duration_notes.append('')

temoa_storageduration = pd.DataFrame({
    'regions':regions,
    'tech':tech,
    'duration':duration,
    'duration_notes':duration_notes
})
temoa_tables_dict['StorageDuration'] = temoa_storageduration

### TechInputSplit

### TechInputSplitAverage

### TechOutputSplit

## Temoa Specific Items:

Add dummy generators to contribute (or remove) firm capacity provided by inter-regional transmission technologies.

Note: this will be scenario specific (i.e. it'll depend on the regions and their aggregations).

References used to make these assumptions:
- [NERC 2022 Summer Reliability Assessment: provides firm capacities](https://www.nerc.com/pa/RAPA/ra/Reliability%20Assessments%20DL/NERC_SRA_2022.pdf)
- [PowerGenome wiki talks about region mappings](https://github.com/PowerGenome/PowerGenome/wiki/Geospatial-Mappings)

In [83]:
# technologies
# efficiency
# existing capacity
# lifetime
# capacitycredit
# variable cost (^)

# _tables = ['technologies', 'Efficiency', 'ExistingCapacity','LifetimeTech','CapacityCredit','CostVariable']
# _regions = [r for r in temoa_tables_dict['PlanningReserveMargin'].regions]
# _regions = [r.split('+')[0] for r in _regions]


# #technologies
# table = 'technologies'
# temoa_tables_dict[table].loc[len(temoa_tables_dict[table])] = \
# ['dummy_capacity','p','electricity','','']

# # efficiency
# table = 'efficiency'
# for r in _regions:
#     temoa_tables_dict[table].loc[len(temoa_tables_dict[table])] = \
#     [_r,'ethos','dummy_capacity',DEFAULT_VINTAGE,'electricity','p',0.1,'Not a real technology. Only provides firm capacity']

# table = 'ExistingCapacity'
# for r in _regions:
#     temoa_tables_dict[table].loc[len(temoa_tables_dict[table])] = \
#     [_r,'dummy_capacity',DEFAULT_VINTAGE,0.01,'GW','']
    
    
# table = 'LifetimeTech'
# for r in _regions:
#     temoa_tables_dict[table].loc[len(temoa_tables_dict[table])] = \
#     [_r,'dummy_capacity',200,'']

# table = 'CapacityCredit'
# for r in _regions:
#     temoa_tables_dict[table].loc[len(temoa_tables_dict[table])] = \
#     [_r,'dummy_capacity',200,'']


In [84]:
list(pg_existing_gens.Resource.unique()) + [t for t in all_techs if 'water_import' in t] 

['BASN_biomass_2',
 'BASN_conventional_hydroelectric_1',
 'BASN_conventional_steam_coal_1',
 'BASN_conventional_steam_coal_2',
 'BASN_geothermal_1',
 'BASN_natural_gas_fired_combined_cycle_1',
 'BASN_natural_gas_fired_combined_cycle_2',
 'BASN_natural_gas_fired_combustion_turbine_1',
 'BASN_natural_gas_fired_combustion_turbine_2',
 'BASN_natural_gas_internal_combustion_engine_1',
 'BASN_natural_gas_steam_turbine_1',
 'BASN_onshore_wind_turbine_1',
 'BASN_petroleum_liquids_1',
 'BASN_small_hydroelectric_1',
 'BASN_solar_photovoltaic_1',
 'CANO_batteries_1',
 'CANO_biomass_1',
 'CANO_biomass_2',
 'CANO_conventional_hydroelectric_1',
 'CANO_hydroelectric_pumped_storage_1',
 'CANO_natural_gas_fired_combined_cycle_1',
 'CANO_natural_gas_fired_combined_cycle_2',
 'CANO_natural_gas_fired_combustion_turbine_1',
 'CANO_natural_gas_fired_combustion_turbine_2',
 'CANO_natural_gas_internal_combustion_engine_1',
 'CANO_onshore_wind_turbine_1',
 'CANO_petroleum_liquids_1',
 'CANO_small_hydroelectric

### Retirement Logic

In [85]:
pg_existing_gens = pd.read_csv(results_directory + 'existing_gens.csv')
pg_existing_gens = pg_existing_gens[pg_existing_gens['Existing_Cap_MW'] >= 100]


temoa_tables_dict['tech_retirement'] = pd.DataFrame({
    'tech':list(pg_existing_gens.Resource.unique()) + [t for t in all_techs if 'water_import' in t],
                  'notes':[' ']*len(list(pg_existing_gens.Resource.unique()) + [t for t in all_techs if 'water_import' in t])})


regions = []
periods = []
tech = []
maxcap = []
mincap = []
maxcap_units = []
maxcap_notes = []
mincap_units = []
mincap_notes = []
primary_keys = set() # (r, p, t)

        
# years = pg_existing_gens.model_year.unique()
# print(years)
# _df = pg_existing_gens
# for idx, row in _df[_df.model_year == years[0]].iterrows():
#     _dff = _df[(_df.Resource == row.Resource) & (_df.region == row.region)]
#     for y in years[1:]:
#         row2 = _dff[_dff.model_year == y]
#         if row2.empty:
#             _mc = 0
#         else:
#             _mc = row2.Existing_Cap_MW.values[0]/1000    
#         regions.append(row.region)
#         tech.append(row.Resource)
#         periods.append(y)
#         maxcap.append(_mc)
#         mincap.append(_mc*0.99)
#         maxcap_notes.append('')
#         maxcap_units.append('GW')
#         if row.technology == 'Conventional Hydroelectric':
#             regions.append(row.region)
#             tech.append('water_import_' + row.Resource)
#             periods.append(y)
#             maxcap.append(_mc)
#             mincap.append(_mc*0.99)
#             maxcap_notes.append('')
#             maxcap_units.append('GW')
print(results_directory + 'existing_gens.csv')
years = pg_existing_gens.model_year.unique()

_df = pg_existing_gens #[pg_existing_gens.model_year > years[0]]
for idx, row in _df.iterrows():
    _mc = row.Existing_Cap_MW  
    if _mc < MIN_CAP_THRESHOLD:
        continue
    regions.append(row.region)
    tech.append(row.Resource)
    periods.append(row.model_year)
    maxcap.append(_mc)
    mincap.append(_mc*0.99)
    maxcap_notes.append('')
    maxcap_units.append('MW')
    mincap_notes.append('')
    mincap_units.append('MW')
    if row.technology == 'Conventional Hydroelectric':
        regions.append(row.region)
        tech.append('water_import_' + row.Resource)
        periods.append(row.model_year)
        maxcap.append(_mc)
        mincap.append(_mc*0.99)
        maxcap_notes.append('')
        maxcap_units.append('MW')
        mincap_notes.append('')
        mincap_units.append('MW')
        
temoa_mincapacity_retirement = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'mincap':mincap,
    'mincap_units':mincap_units,
    'mincap_notes':mincap_notes
})

# Above logic only covers cases for which MW > 0
# Below we set MaxCap = 0 for rows that don't exist
_gens = pg_existing_gens.Resource.unique()
_df0 = pg_existing_gens[pg_existing_gens.model_year == years[0]]
for y in years[1:]:
    _df = pg_existing_gens[pg_existing_gens.model_year == y]
    for _g in _gens:
        if _g in _df.Resource.unique():
            continue
        for idx, row in _df0[_df0.Resource == _g].iterrows():
            regions.append(row.region)
            tech.append(row.Resource)
            periods.append(y)
            maxcap.append(0)
            maxcap_notes.append('')
            maxcap_units.append('MW')
            if row.technology == 'Conventional Hydroelectric':
                regions.append(row.region)
                tech.append('water_import_' + row.Resource)
                periods.append(row.model_year)
                maxcap.append(0)
                maxcap_notes.append('')
                maxcap_units.append('MW')
            break
        
        
        
        

    
temoa_maxcapacity_retirement = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'tech':tech,
    'maxcap':maxcap,
    'maxcap_units':maxcap_units,
    'maxcap_notes':maxcap_notes
})

temoa_maxcapacity_retirement[temoa_maxcapacity_retirement.tech == 'BASN_batteries_1']


/mnt/c/Users/camer/Documents/CMU/Modelling/PowerGenome/Git-Results-Comparison/MIP_results_comparison/case_settings/26-zone/settings-atb2023/Computed_Data_base_20_week/existing_gens.csv


,regions,periods,tech,maxcap,maxcap_units,maxcap_notes


In [86]:
temoa_mincapacity_retirement

,regions,periods,tech,mincap,mincap_units,mincap_notes
0,BASN,2027,BASN_conventional_hydroelectric_1,2464.62480,MW,
1,BASN,2027,water_import_BASN_conventional_hydroelectric_1,2464.62480,MW,
2,BASN,2027,BASN_conventional_steam_coal_1,1166.41800,MW,
3,BASN,2027,BASN_conventional_steam_coal_2,1803.78000,MW,
4,BASN,2027,BASN_geothermal_1,416.04156,MW,
...,...,...,...,...,...,...
1812,SRCE,2050,SRCE_distributed_generation_1,3971.88000,MW,
1813,SRSE,2050,SRSE_distributed_generation_1,3079.89000,MW,
1814,SRSG,2050,SRSG_distributed_generation_1,3248.19000,MW,
1815,TRE,2050,TRE_distributed_generation_1,9492.12000,MW,


In [87]:
_df = temoa_mincapacity_retirement
for r in _df.regions.unique():
    for t in _df[_df.regions == r].tech.unique():
        xx = _df[(_df.regions == r) & (_df.tech == t)]

        if (xx['mincap'].diff() > 0).any():
            print("There are ascending mincaps in the dataframe: ", r, t)


There are ascending mincaps in the dataframe:  BASN BASN_distributed_generation_1
There are ascending mincaps in the dataframe:  CANO CANO_distributed_generation_1
There are ascending mincaps in the dataframe:  CASO CASO_distributed_generation_1
There are ascending mincaps in the dataframe:  FRCC FRCC_distributed_generation_1
There are ascending mincaps in the dataframe:  ISNE ISNE_distributed_generation_1
There are ascending mincaps in the dataframe:  MISC MISC_distributed_generation_1
There are ascending mincaps in the dataframe:  MISE MISE_distributed_generation_1
There are ascending mincaps in the dataframe:  MISS MISS_distributed_generation_1
There are ascending mincaps in the dataframe:  MISW MISW_distributed_generation_1
There are ascending mincaps in the dataframe:  NWPP NWPP_distributed_generation_1
There are ascending mincaps in the dataframe:  NYCW NYCW_distributed_generation_1
There are ascending mincaps in the dataframe:  NYUP NYUP_distributed_generation_1
There are ascend

In [88]:

_df = temoa_maxcapacity
for r in _df.regions.unique():
    for t in _df[_df.regions == r].tech.unique():
        xx = _df[(_df.regions == r) & (_df.tech == t)]

        if (xx['maxcap'].diff() < 10).any():
            print(xx['maxcap'].diff())
            print("There are ascending maxcaps in the dataframe: ", r, t)

96       NaN
1110    10.0
2124    10.0
3138    10.0
4152    10.0
5166    10.0
Name: maxcap, dtype: float64
There are ascending maxcaps in the dataframe:  FRCC FRCC_landbasedwind_class3_moderate_4_anyQual_1
107      NaN
1121    10.0
2135    10.0
3149    10.0
4163    10.0
5177    10.0
Name: maxcap, dtype: float64
There are ascending maxcaps in the dataframe:  FRCC FRCC_landbasedwind_class3_moderate_15_anyQual_1
127      NaN
1141    10.0
2155    10.0
3169    10.0
4183    10.0
5197    10.0
Name: maxcap, dtype: float64
There are ascending maxcaps in the dataframe:  FRCC FRCC_utilitypv_class1_moderate_11_anyQual_1
159      NaN
1173    10.0
2187    10.0
3201    10.0
4215    10.0
5229    10.0
Name: maxcap, dtype: float64
There are ascending maxcaps in the dataframe:  ISNE ISNE_utilitypv_class1_moderate_14_anyQual_0
182      NaN
1196    10.0
2210    10.0
3224    10.0
4238    10.0
5252    10.0
Name: maxcap, dtype: float64
There are ascending maxcaps in the dataframe:  ISNE ISNE_offshorewind_clas

In [89]:
temoa_maxcapacity_retirement[temoa_maxcapacity_retirement.tech.str.contains('distributed', na=False)]

,regions,periods,tech,maxcap,maxcap_units,maxcap_notes
307,BASN,2027,BASN_distributed_generation_1,1193.0,MW,
308,CANO,2027,CANO_distributed_generation_1,4478.0,MW,
309,CASO,2027,CASO_distributed_generation_1,6969.0,MW,
310,FRCC,2027,FRCC_distributed_generation_1,2946.0,MW,
311,ISNE,2027,ISNE_distributed_generation_1,4187.0,MW,
...,...,...,...,...,...,...
1812,SRCE,2050,SRCE_distributed_generation_1,4012.0,MW,
1813,SRSE,2050,SRSE_distributed_generation_1,3111.0,MW,
1814,SRSG,2050,SRSG_distributed_generation_1,3281.0,MW,
1815,TRE,2050,TRE_distributed_generation_1,9588.0,MW,


In [90]:
if 'MinCapacity' in temoa_tables_dict.keys():
    temoa_tables_dict['MinCapacity'] = pd.concat([temoa_tables_dict['MinCapacity'], temoa_mincapacity_retirement])
else:
    temoa_tables_dict['MinCapacity'] = temoa_mincapacity_retirement
    
if 'MaxCapacity' in temoa_tables_dict.keys():
    temoa_tables_dict['MaxCapacity'] = pd.concat([temoa_tables_dict['MaxCapacity'], temoa_maxcapacity_retirement])
else:
    temoa_tables_dict['MaxCapacity'] = temoa_maxcapacity_retirement

In [91]:
temoa_maxcapacity_retirement[temoa_maxcapacity_retirement.tech == 'PJMW_natural_gas_fired_combustion_turbine_1']

,regions,periods,tech,maxcap,maxcap_units,maxcap_notes
181,PJMW,2027,PJMW_natural_gas_fired_combustion_turbine_1,10942.446,MW,
511,PJMW,2030,PJMW_natural_gas_fired_combustion_turbine_1,10942.446,MW,
830,PJMW,2035,PJMW_natural_gas_fired_combustion_turbine_1,10942.446,MW,
1139,PJMW,2040,PJMW_natural_gas_fired_combustion_turbine_1,10942.446,MW,
1430,PJMW,2045,PJMW_natural_gas_fired_combustion_turbine_1,10182.490,MW,
1697,PJMW,2050,PJMW_natural_gas_fired_combustion_turbine_1,8602.680,MW,


### Distributed Generation
In Powergenome, DG appears as an existing technology in the year in question. 
We need to redefine the technology as a resource option and set its MinCap=MaxCap

The MinCap=MaxCap constraint is handled above in the Retirement Logic section. So here, we need to redefine its efficiency and capacity factor.

Note: 

In [92]:
dfe = temoa_efficiency[temoa_efficiency.tech.str.contains('distributed')]
for year in pg_new_gens.model_year.unique():
    if year in dfe.vintage.unique():
        continue
    else:
        dfe.vintage = year
        temoa_efficiency = pd.concat([temoa_efficiency, dfe], ignore_index=True)
    
#dfe = temoa_efficiency[temoa_efficiency.tech == 'WEST_distributed_generation_1']
#for year in pg_new_gens.model_year.unique():
#    if year in dfe.vintage.unique():
#        continue
#    else:
#        dfe.vintage = year
#        temoa_efficiency = pd.concat([temoa_efficiency, dfe], ignore_index=True)
        
temoa_tables_dict['Efficiency'] = temoa_efficiency #= temoa_efficiency.reindex()


/home/cam/mambaforge/envs/powergenome/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [93]:
tech=[]
flag=[]
sector=[]
tech_desc=[]
tech_category=[]

for t in all_techs:
    tech.append(t)
    flag.append('ps' if t in techs_storage else 'p')
    if 'import' in t:
        _s = 'import'
    elif 'distribution' in t:
        _s = 'distribution'
    elif 'transmission' in t:
        _s = 'transmission'
    else:
        _s = 'electricity'
    sector.append(_s)
    tech_desc.append('')
    tech_category.append('')
    

temoa_technologies = pd.DataFrame({'tech':tech,
                                   'flag':flag,
                                   'sector':sector,
                                   'tech_desc':tech_desc,
                                   'tech_category':tech_category})
temoa_tables_dict['technologies'] = temoa_technologies

### Include DAC

In [94]:
if True:
    dac_region = list(pg_new_gens.region.unique())[0]
    temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
    ['CO2_Offset','p','dummy','','This technology allows CO2 offsets at a fixed price'] 
    temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
    ['Dummy_Offset','p','dummy','','Dummy technology to meet dummy CO2 Offset demand'] 


    temoa_tables_dict['commodities'].loc[len(temoa_tables_dict['commodities'])] = \
    ['Dummy_CO2_Offset_Demand','d','Dummy demand for captured/offset CO2']
    
    temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
    [dac_region,'CO2_Offset',8760.0,''] 
    temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
    [dac_region,'Dummy_Offset',8760.0,''] 
    

    temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
    [dac_region,'CO2_Offset',100,''] 

    temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
    [dac_region,'Dummy_Offset',100,''] 
    
    
    temoa_tables_dict['ExistingCapacity'].loc[len(temoa_tables_dict['ExistingCapacity'])] = \
    [dac_region,'CO2_Offset',DEFAULT_VINTAGE,999999.0,'MW',''] 

    temoa_tables_dict['ExistingCapacity'].loc[len(temoa_tables_dict['ExistingCapacity'])] = \
    [dac_region,'Dummy_Offset',DEFAULT_VINTAGE,999999.0,'MW',''] 

    
    
    temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
    [dac_region,'ethos', 'CO2_Offset', DEFAULT_VINTAGE, 'Dummy_CO2_Offset_Demand', 1.0,''] 

    temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
    [dac_region,'ethos', 'Dummy_Offset', DEFAULT_VINTAGE, 'Dummy_CO2_Offset_Demand', 1.0,''] 


    temoa_tables_dict['EmissionActivity'].loc[len(temoa_tables_dict['EmissionActivity'])] = \
    [dac_region,'CO2','ethos', 'CO2_Offset', DEFAULT_VINTAGE, 'Dummy_CO2_Offset_Demand', -1.0,'',''] 

    for y in pg_new_gens.model_year.unique():
        temoa_tables_dict['CostVariable'].loc[len(temoa_tables_dict['CostVariable'])] = \
        [dac_region,y, 'CO2_Offset', DEFAULT_VINTAGE, 200.0, '$/t',''] 


        temoa_tables_dict['Demand'].loc[len(temoa_tables_dict['Demand'])] = \
        [dac_region, y,  'Dummy_CO2_Offset_Demand', 900000000, 't','Should be set to some arbitrarily large number.'] 


In [95]:
# if True:
#     dac_region = list(pg_new_gens.region.unique())[0]
#     temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
#     ['CO2_Offset','p','dummy','','This technology allows CO2 offsets at a fixed price'] 
#     temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
#     ['Dummy_Offset','p','dummy','','Dummy technology to meet dummy CO2 Offset demand'] 


#     temoa_tables_dict['commodities'].loc[len(temoa_tables_dict['commodities'])] = \
#     ['Dummy_CO2_Offset_Demand','d','Dummy demand for captured/offset CO2']
    
#     temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
#     [dac_region,'CO2_Offset',8760.0,''] 
#     temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
#     [dac_region,'Dummy_Offset',8760.0,''] 
    

#     temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
#     [dac_region,'CO2_Offset',100,''] 

#     temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
#     [dac_region,'Dummy_Offset',100,''] 

#     for year in pg_new_gens.model_year.unique():
    
#         temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
#         [dac_region,'ethos', 'CO2_Offset', year, 'Dummy_CO2_Offset_Demand', 1.0,''] 

#         temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
#         [dac_region,'ethos', 'Dummy_Offset', year, 'Dummy_CO2_Offset_Demand', 1.0,''] 


#         temoa_tables_dict['EmissionActivity'].loc[len(temoa_tables_dict['EmissionActivity'])] = \
#         [dac_region,'CO2','ethos', 'CO2_Offset', year, 'Dummy_CO2_Offset_Demand', -1.0,'',''] 
        
#         for y in pg_new_gens.model_year.unique():
#             if y >= year:
#                 temoa_tables_dict['CostVariable'].loc[len(temoa_tables_dict['CostVariable'])] = \
#                 [dac_region,y, 'CO2_Offset', year, 200.0, '$/',''] 

#         temoa_tables_dict['Demand'].loc[len(temoa_tables_dict['Demand'])] = \
#         [dac_region,year,  'Dummy_CO2_Offset_Demand', 500000000, 't','Should be set to some arbitrarily large number.'] 


### Include unserved load

In [96]:
# temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
#     ['unserved_load','p','dummy','','This tech meets load at the cost of unserved energy'] 

# for _r in pg_new_gens.region.unique():

#     temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
#     [_r,'unserved_load',200,''] 

#     temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
#     [_r,'unserved_load',8760.0,''] 
    
#     for _v in planning_periods:
#         temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
#         [_r,'ethos', 'unserved_load', _v, 'electricity', 1.0,''] 
#         temoa_tables_dict['MaxCapacity'].loc[len(temoa_tables_dict['MaxCapacity'])] = \
#         [_r,_v, 'unserved_load', 1000000,'MW',''] 
#         for _p in planning_periods:
#             if _p >= _v:
#                 temoa_tables_dict['CostVariable'].loc[len(temoa_tables_dict['CostVariable'])] = \
#                 [_r,_p, 'unserved_load', _v, 5000.0, '$/MWh',''] 


In [97]:
temoa_tables_dict['technologies'].loc[len(temoa_tables_dict['technologies'])] = \
    ['unserved_load','p','dummy','','This tech meets load at the cost of unserved energy'] 

for _r in pg_new_gens.region.unique():

    temoa_tables_dict['LifetimeTech'].loc[len(temoa_tables_dict['LifetimeTech'])] = \
    [_r,'unserved_load',200,''] 

    temoa_tables_dict['CapacityToActivity'].loc[len(temoa_tables_dict['CapacityToActivity'])] = \
    [_r,'unserved_load',8760.0,''] 
    
    temoa_tables_dict['ExistingCapacity'].loc[len(temoa_tables_dict['ExistingCapacity'])] = \
    [_r,'unserved_load',DEFAULT_VINTAGE,10000.0,'MW',''] 
    
    
    temoa_tables_dict['Efficiency'].loc[len(temoa_tables_dict['Efficiency'])] = \
    [_r,'ethos', 'unserved_load', DEFAULT_VINTAGE, 'electricity', 1.0,''] 
    

    
    for _p in planning_periods:
        temoa_tables_dict['CostVariable'].loc[len(temoa_tables_dict['CostVariable'])] = \
        [_r,_p, 'unserved_load', DEFAULT_VINTAGE, 5000.0, '$/MWh',''] 

### RPS and CES

In [98]:
pg_policies = pd.read_csv(results_directory + 'policies.csv')
#pg_policies = pg_policies[(pg_policies.case_id == scenario) & (pg_policies.year.isin(pg_demand.year.unique()))]


In [99]:
region = []
group_name = []
tech = []
notes = []

ESR_cols = [x for x in pg_policies.columns if 'ESR' in x]

for esr in ESR_cols:
    _df = pg_all_gens[pg_all_gens[esr] == 1]
    for idx, row in _df.iterrows():
        region.append(row.region)
        group_name.append(esr)
        tech.append(row.Resource)
        notes.append('')

temoa_techgroups = pd.DataFrame({'region':region,
                                   'group_name':group_name,
                                   'tech':tech,
                                   'notes':notes})
temoa_tables_dict['tech_groups'] = temoa_techgroups
temoa_techgroups

,region,group_name,tech,notes
0,BASN,ESR_1,BASN_biomass_1,
1,BASN,ESR_1,BASN_biomass_2,
2,BASN,ESR_1,BASN_geothermal_1,
3,BASN,ESR_1,BASN_onshore_wind_turbine_1,
4,BASN,ESR_1,BASN_small_hydroelectric_1,
...,...,...,...,...
14666,TREW,ESR_16,TREW_utilitypv_class1_moderate_5_anyQual_1,
14667,TREW,ESR_16,TREW_utilitypv_class1_moderate_6_anyQual_0,
14668,TREW,ESR_16,TREW_utilitypv_class1_moderate_7_anyQual_1,
14669,TREW,ESR_16,TREW_utilitypv_class1_moderate_8_anyQual_0,


In [100]:
temoa_tables_dict['groups'] = pd.DataFrame(columns=["group_name", "notes"])
group_name = []
notes = []
for esr in ESR_cols:
    temoa_tables_dict['groups'].loc[len(temoa_tables_dict['groups'])] = \
    [esr,''] 

In [101]:
pg_policies

,Unnamed: 0,case_id,year,region,ESR_1,ESR_2,ESR_3,ESR_4,ESR_5,ESR_6,...,ESR_11,ESR_12,ESR_13,ESR_14,ESR_15,ESR_16,CO_2_Cap_Zone_1,CO_2_Max_Mtons_1,copy_case_id,Note
0,0,all,2027,BASN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,873.0,NaN,halfway between 2023 (1560 MMT according to EP...
1,1,all,2027,CANO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2,all,2027,CASO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,3,all,2027,FRCC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,4,all,2027,ISNE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,151,all,2050,SRCE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
152,152,all,2050,SRSE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
153,153,all,2050,SRSG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
154,154,all,2050,TRE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [102]:
ESR_cols = [x for x in pg_policies.columns if 'ESR' in x]
ESR_demands = {}
for y in pg_policies.year.unique():
    ESR_demands[y] = {}
    _pg_policies = pg_policies[pg_policies.year == y]
    print(y)
    for esr in ESR_cols:
        _dem = 0  # Reset _dem to 0 for each ESR_col
        if _pg_policies[esr].sum() == 0:
            continue
        for idx, row in _pg_policies.iterrows():
            if row[esr] < 0.01:
                continue
            _dem += row[esr] * temoa_demand[(temoa_demand.periods == y) & (temoa_demand.regions == row.region) & (temoa_demand.demand_comm == 'DEMAND_ELC')].demand.values[0]
        ESR_demands[y][esr] = _dem  # Directly store the float, as it's immutable



2027
2030
2035
2040
2045
2050


In [103]:
ESR_cols = [x for x in pg_policies.columns if 'ESR' in x]
ESR_regions = {}
for esr in ESR_cols:
    ESR_regions[esr] = list(pg_all_gens[pg_all_gens[esr] == 1].region.unique())

In [104]:
ESR_regions

{'ESR_1': ['BASN', 'CANO', 'CASO', 'NWPP', 'RMRG', 'SRSG'],
 'ESR_2': ['ISNE'],
 'ESR_3': ['MISC', 'MISE', 'MISS', 'MISW'],
 'ESR_4': ['NYCW', 'NYUP'],
 'ESR_5': ['PJMC', 'PJMD', 'PJME', 'PJMW'],
 'ESR_6': ['SPPC', 'SPPN', 'SPPS'],
 'ESR_7': ['FRCC', 'SRCA', 'SRCE', 'SRSE'],
 'ESR_8': ['TRE', 'TREW'],
 'ESR_9': ['BASN', 'CANO', 'CASO', 'NWPP', 'RMRG', 'SRSG'],
 'ESR_10': ['ISNE'],
 'ESR_11': ['MISC', 'MISE', 'MISS', 'MISW'],
 'ESR_12': ['NYCW', 'NYUP'],
 'ESR_13': ['PJMC', 'PJMD', 'PJME', 'PJMW'],
 'ESR_14': ['SPPC', 'SPPN', 'SPPS'],
 'ESR_15': ['FRCC', 'SRCA', 'SRCE', 'SRSE'],
 'ESR_16': ['TRE', 'TREW']}

In [105]:

# Define the column names
columns = ["regions", "periods", "group_name", "min_act_g", "notes"]
regions = []
periods = []
group_name = []
min_act_g = []
notes = []

for y in ESR_demands.keys():
    for esr in ESR_demands[y].keys():
        regions.append('+'.join(ESR_regions[esr]))
        periods.append(y)
        group_name.append(esr)
        min_act_g.append(ESR_demands[y][esr])
        notes.append('RPS')
        


temoa_minactivitygroup = pd.DataFrame({
    'regions':regions,
    'periods':periods,
    'group_name':group_name,
    'min_act_g': min_act_g,
    'notes':notes
})
temoa_tables_dict['MinActivityGroup'] = temoa_minactivitygroup
temoa_minactivitygroup

,regions,periods,group_name,min_act_g,notes


In [106]:
temoa_existingcapacity

,regions,tech,vintage,exist_cap,exist_cap_units,exist_cap_notes
0,BASN,BASN_biomass_2,2000,16.776,MW,
1,BASN,BASN_conventional_hydroelectric_1,2000,2489.520,MW,
2,BASN,BASN_conventional_steam_coal_1,2000,1178.200,MW,
3,BASN,BASN_conventional_steam_coal_2,2000,1822.000,MW,
4,BASN,BASN_geothermal_1,2000,420.244,MW,
...,...,...,...,...,...,...
731,SRCE,unserved_load,2000,10000.000,MW,
732,SRSE,unserved_load,2000,10000.000,MW,
733,SRSG,unserved_load,2000,10000.000,MW,
734,TRE,unserved_load,2000,10000.000,MW,


In [107]:
for name, table in temoa_tables_dict.items():
    if isinstance(table, pd.DataFrame):
        temoa_tables_dict[name] = table.drop_duplicates()

# Construct SQLite File

In [ ]:
import shutil
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, text

# This is the fastest, and most memory-efficient option.
# However, downstream I/O operations in Temoa pformat_results take ages.
# def populate_sqlite_from_dict(template_file, tables_dict, output_file, chunk_size=10000):
#     # Ensure output directory exists
#     output_dir = os.path.dirname(output_file)
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     # Copy the template file to the new SQLite file
#     shutil.copy(template_file, output_file)

#     # Create a connection to the SQLite file
#     engine = create_engine(f'sqlite:///{output_file}')
    
#     with engine.connect() as conn:
#         # Set PRAGMA settings for optimization
#         conn.execute(text('PRAGMA synchronous = OFF'))
#         conn.execute(text('PRAGMA journal_mode = OFF'))
#         conn.execute(text('PRAGMA cache_size = -100000'))  # Set cache size in KB
#         conn.execute(text('PRAGMA locking_mode = EXCLUSIVE'))
        
#         # Reflect the existing schema
#         metadata = MetaData()
#         metadata.reflect(bind=engine)
        
#         # Start a transaction
#         with conn.begin():
#             for table, data in tables_dict.items():
#                 print(table)
#                 # Convert the value to a DataFrame if it's a dictionary
#                 if isinstance(data, dict):
#                     df = pd.DataFrame.from_dict(data)
#                 elif isinstance(data, pd.DataFrame):
#                     df = data
#                 else:
#                     raise ValueError(f"Unsupported data type for table '{table}'.")

#                 # Ensure the DataFrame columns match the existing table schema
#                 if table in metadata.tables:
#                     existing_table = metadata.tables[table]
#                     df = df[existing_table.columns.keys()]

#                 # Write the data to the table in chunks without altering the schema
#                 df.to_sql(table, con=conn, if_exists='append', index=False, chunksize=chunk_size)

#     # Vacuum the database after all insertions are complete
#     with engine.connect() as conn:
#         conn.execute(text('VACUUM'))





def populate_sqlite_from_dict(template_file, tables_dict, output_file, chunk_size=10000):
    # Ensure output directory exists
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Copy the template file to the new SQLite file
    shutil.copy(template_file, output_file)

    # Create a connection to the SQLite file
    engine = create_engine(f'sqlite:///{output_file}')
    
    with engine.connect() as conn:
        # Set PRAGMA settings for optimization
        conn.execute(text('PRAGMA synchronous = OFF'))
        conn.execute(text('PRAGMA journal_mode = OFF'))
        conn.execute(text('PRAGMA cache_size = -100000'))  # Set cache size in KB
        conn.execute(text('PRAGMA locking_mode = EXCLUSIVE'))

        
        # Reflect the existing schema
        metadata = MetaData()
        metadata.reflect(bind=engine)

        # Separate the CapacityFactorTech entry to process it last
        tables_dict_ordered = {k: v for k, v in tables_dict.items() if k != "CapacityFactorTech"}
        if "CapacityFactorTech" in tables_dict:
            tables_dict_ordered["CapacityFactorTech"] = tables_dict["CapacityFactorTech"]
        
        # Start a transaction
        with conn.begin():
            for table, data in tables_dict_ordered.items():
                print(f"Processing table: {table}")
                # Convert the value to a DataFrame if it's a dictionary
                if isinstance(data, dict):
                    df = pd.DataFrame.from_dict(data)
                elif isinstance(data, pd.DataFrame):
                    df = data
                else:
                    raise ValueError(f"Unsupported data type for table '{table}'.")

                # Rename columns if necessary
                if table == "CapacityToActivity":
                    df.rename(columns={"c2a_tech": "c2a", "c2a_tech_notes": "c2a_notes"}, inplace=True)

                # Ensure the DataFrame columns match the existing table schema
                if table in metadata.tables:
                    existing_table = metadata.tables[table]
                    existing_columns = existing_table.columns.keys()
                    print(f"Expected columns for table {table}: {existing_columns}")
                    missing_columns = [col for col in existing_columns if col not in df.columns]
                    if missing_columns:
                        print(f"Missing columns for table {table}: {missing_columns}")
                        # Optionally, add missing columns with default values (e.g., None or 0)
                        for col in missing_columns:
                            df[col] = None  # or an appropriate default value

                # Write the data to the table in chunks without altering the schema
                df.to_sql(table, con=conn, if_exists='append', index=False, chunksize=chunk_size)

#     # Vacuum the database after all insertions are complete
#     with engine.connect() as conn:
#         conn.execute(text('VACUUM'))



# def populate_sqlite_from_dict(template_file, tables_dict, output_file, chunk_size=20000):
#     # Ensure output directory exists
#     output_dir = os.path.dirname(output_file)
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     # Copy the template file to the new SQLite file
#     shutil.copy(template_file, output_file)

#     # Create a connection to the SQLite file
#     engine = create_engine(f'sqlite:///{output_file}')
    
#     with engine.connect() as conn:
#         # Set PRAGMA settings for optimization
#         conn.execute(text('PRAGMA synchronous = OFF'))
#         conn.execute(text('PRAGMA journal_mode = OFF'))
#         conn.execute(text('PRAGMA cache_size = -100000'))  # Set cache size in KB
#         conn.execute(text('PRAGMA locking_mode = EXCLUSIVE'))
#         conn.execute(text('PRAGMA foreign_keys = OFF'))  # Disable foreign keys

#         # Reflect the existing schema
#         metadata = MetaData()
#         metadata.reflect(bind=engine)
        
#         # Separate the CapacityFactorTech entry to process it last
#         tables_dict_ordered = {k: v for k, v in tables_dict.items() if k != "CapacityFactorTech"}
#         if "CapacityFactorTech" in tables_dict:
#             tables_dict_ordered["CapacityFactorTech"] = tables_dict["CapacityFactorTech"]
        
#         # Start a transaction
#         with conn.begin():
#             print('Made it here.')
#             for table, data in tables_dict_ordered.items():
#                 print(table)
#                 print(f"Processing table: {table}")
#                 # Convert the value to a DataFrame if it's a dictionary
#                 if isinstance(data, dict):
#                     df = pd.DataFrame.from_dict(data)
#                 elif isinstance(data, pd.DataFrame):
#                     df = data
#                 else:
#                     raise ValueError(f"Unsupported data type for table '{table}'.")

#                 # Ensure the DataFrame columns match the existing table schema
#                 if table in metadata.tables:
#                     existing_table = metadata.tables[table]
#                     existing_columns = existing_table.columns.keys()
#                     print(f"Expected columns for table {table}: {existing_columns}")
#                     missing_columns = [col for col in existing_columns if col not in df.columns]
#                     if missing_columns:
#                         print(f"Missing columns for table {table}: {missing_columns}")
#                         # Optionally, add missing columns with default values (e.g., None or 0)
#                         for col in missing_columns:
#                             df[col] = None  # or an appropriate default value

#                 # Write the data to the table in chunks without altering the schema
# #                 if table == "CapacityFactorTech":
# #                     df.to_sql(table, con=conn, if_exists='append', index=False, chunksize=chunk_size, method='multi')
# #                 else:
#                 df.to_sql(table, con=conn, if_exists='append', index=False, chunksize=chunk_size)

#         # Re-enable foreign keys and indexes
#         # conn.execute(text('PRAGMA foreign_keys = ON'))

#     # Vacuum the database after all insertions are complete
#     print('Vacuuming...')
#     with engine.connect() as conn:
#         conn.execute(text('VACUUM'))

In [ ]:

# Assuming you have the variables defined as follows:
if '52' in scenario:
    template_file = '/mnt/c/Users/camer/Documents/CMU/Modelling/Temoa/temoa/data_files/temoa_schema_52.sqlite'
else:
    template_file = '/mnt/c/Users/camer/Documents/CMU/Modelling/Temoa/temoa/data_files/temoa_schema.sqlite'
fname = sql_directory + scenario + 'build.sqlite'

populate_sqlite_from_dict(template_file, temoa_tables_dict, fname)

In [ ]:
temoa_tables_dict['ExistingCapacity']

In [ ]:
temoa_tables_dict['CostFixed'].rename(columns={"cost_fixed_tech": "cost_fixed"}, inplace=True)



# Print To SQL

In [108]:
fname = sql_directory + scenario + '.sql'
eps = 0.00001
print(eps)

1e-05


In [109]:
print('test')
with open(fname, 'w+') as outfile:
    outfile.write("BEGIN TRANSACTION;\n")
    for name, table in temoa_tables_dict.items():
        print(name)
        if type(table) == pd.core.frame.DataFrame:
            table = table.drop_duplicates()
            cols = table.columns
            for idx, row in table.iterrows():
                _str = "INSERT INTO '" + \
                    str(name) + "' VALUES ("
                for col in cols:
                    if col != cols[0]:
                        _str += ", "
                    if type(row[col]) == str:
                        _str += "'" + row[col] + "'"
                    else:
                        if (abs(row[col])) < eps:
                            val = str(0.0)
                        else:
                            val = str(row[col]) 
                        _str += val 
                _str += ");\n"
                outfile.write(_str)
        else: # dictionary
            for key, val in table.items():
                for v in val:
                    _str = "INSERT INTO '" + \
                        str(name) + "' VALUES ('" + str(v) + "');\n"
                    outfile.write(_str)
    outfile.write("COMMIT;\n")        

test
time_periods
time_of_day
time_season
time_seasons_per_period
regions
technologies
tech_curtailment
tech_exchange
tech_reserve
commodities
groups
Efficiency
LifetimeTech
CapacityCredit
CapacityFactorTech
CapacityToActivity
CostFixed
CostInvest
CostVariable
Demand
DemandSpecificDistribution
DiscountRate
EmissionActivity
EmissionLimit
region_combinations
ExistingCapacity
GlobalDiscountRate
LifetimeLoanTech
MaxCapacity
MaxNewCapacity
MinAnnualCapacityFactor
PlanningReserveMargin
SegFrac
StorageDuration
tech_retirement
MinCapacity
tech_groups
MinActivityGroup


# Construct Script to Compile the SQLite file

In [110]:
if '52' in scenario:
    template = '/mnt/c/Users/camer/Documents/CMU/Modelling/Temoa/temoa/data_files/temoa_schema_52.sql'
else:
    template = '/mnt/c/Users/camer/Documents/CMU/Modelling/Temoa/temoa/data_files/temoa_schema.sql'


In [111]:
line1 = '#!/bin/sh'
line2 = 'rm ' + scenario + '.sqlite'
line3 = 'sqlite3 ' + scenario + '.sqlite < ' + template
line4 = 'sqlite3 ' + scenario + '.sqlite < ' + scenario + '.sql'
line5 = 'cp ' + scenario + '.sqlite ../Solved_SQLite_Files/'+ scenario + '.sqlite' #'_' + str(current_period) + '.sqlite'
lines = [line1, line2, line3, line4,line5]


In [112]:
scenario

'base_20_week'

In [113]:
with open(sql_directory + 'make_sqlite.sh', 'w+') as text_file:
    for line in lines:
        text_file.write(line + '\n')


In [114]:
temoa_maxnewcapacity

,regions,periods,tech,maxnewcap,maxnewcap_units,maxnewcap_notes
0,BASN-CANO,2027,transmission,10000.0,MW,
1,CANO-BASN,2027,transmission,10000.0,MW,
2,BASN-CANO,2030,transmission,10000.0,MW,
3,CANO-BASN,2030,transmission,10000.0,MW,
4,BASN-CANO,2035,transmission,10000.0,MW,
...,...,...,...,...,...,...
679,TRE-SRSG,2040,transmission,10000.0,MW,
680,SRSG-TRE,2045,transmission,10000.0,MW,
681,TRE-SRSG,2045,transmission,10000.0,MW,
682,SRSG-TRE,2050,transmission,10000.0,MW,


In [115]:
temoa_maxcapacity

,regions,periods,tech,maxcap,maxcap_units,maxcap_notes
0,BASN,2027,BASN_landbasedwind_class3_moderate_0_anyQual_0,8527.208,MW,
1,BASN,2027,BASN_landbasedwind_class3_moderate_1_anyQual_1,8168.107,MW,
2,BASN,2027,BASN_landbasedwind_class3_moderate_2_anyQual_0,9068.288,MW,
3,BASN,2027,BASN_landbasedwind_class3_moderate_3_anyQual_1,7607.149,MW,
4,BASN,2027,BASN_landbasedwind_class3_moderate_4_anyQual_0,9564.075,MW,
...,...,...,...,...,...,...
6079,TREW,2050,TREW_utilitypv_class1_moderate_5_anyQual_1,4334.576,MW,
6080,TREW,2050,TREW_utilitypv_class1_moderate_6_anyQual_0,4545.979,MW,
6081,TREW,2050,TREW_utilitypv_class1_moderate_7_anyQual_1,5437.589,MW,
6082,TREW,2050,TREW_utilitypv_class1_moderate_8_anyQual_0,3930.511,MW,


In [116]:
pg_existing_gens[(pg_existing_gens.technology == 'Biomass') & (pg_existing_gens.region == 'NYCW')]

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,...,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year,lifetime_yrs,end_of_life,Storage_Duration,model_year


In [117]:
temoa_tables_dict['CapacityToActivity'][temoa_tables_dict['CapacityToActivity'].tech == 'SPPC_utilitypv_class1_moderate_9_anyQual_1']

,regions,tech,c2a,c2a_tech_notes
1176,SPPC,SPPC_utilitypv_class1_moderate_9_anyQual_1,8760.0,
